In [68]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
import re
import pickle
import copy

# !pip install -U spacy
# import spacy
# spacy.__version__

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.56 ms (started: 2022-09-09 11:22:59 +00:00)


In [69]:
!pip install openpyxl
import openpyxl
def build_gold_dict():
    # read from excel
    read_file = pd.read_excel('../input/gold-dict/Terms malantin 1er juin 2022.xlsx',
                              sheet_name='categories 1 juin 2022')
    read_file.dropna(0, how='all', inplace=True)
    read_file.dropna(1, how='all', inplace=True)

    gold_dict = {}
    for i in range(1, len(read_file)):
        if read_file.iloc[i]['Main form'] is None:
            continue
        gold_dict[read_file.iloc[i]['Main form'].strip()] = []

    for i in range(1, len(read_file)):
        index = read_file.iloc[i]['Main form'].strip()
        if index is None:
            continue
        if not read_file.iloc[i].isna()['Sustainability preoccupations']:
            gold_dict[index].append('Sustainability preoccupations')
        if not read_file.iloc[i].isna()['Digital transformation']:
            gold_dict[index].append('Digital transformation')
        if not read_file.iloc[i].isna()['Change in management']:
            gold_dict[index].append('Change in management')
        if not read_file.iloc[i].isna()['Innovation activities']:
            gold_dict[index].append('Innovation activities')
        if not read_file.iloc[i].isna()['Business Model']:
            gold_dict[index].append('Business Model')
        if not read_file.iloc[i].isna()['Corporate social responsibility ou CSR']:
            gold_dict[index].append('Corporate social responsibility ou CSR') 

    # Change the category for four keywords
    # academic institutions, university & research institutions, service among university, 
    # worldwide research centres
    category = 'Innovation activities'
    changelist = ['academic institutions', 'worldwide research centers', 'university and research institutions', 'customer service among university']
    for c in changelist:
        gold_dict[c][0] = category

    # Deal with the singular and plural cases in keywords
    cortext3 = open("../input/cortext/Cortext3_min_delac_flex_utf8.txt", "r")
    lines = cortext3.readlines()
    lefts, rights = [], []
    for line in lines:
        left, right = line.split(',')
        if left != right.split('.')[0]:
            lefts.append(left)
            rights.append(right.split('.')[0])

    tmp_gold_dict = gold_dict.copy()
    for key, value in tmp_gold_dict.items():
        if key in lefts:
            index = lefts.index(key)
            right = rights[index]
            gold_dict[right] = gold_dict[key]
        if key in rights:
            indices = [i for i, word in enumerate(rights) if word == key]
            for index in indices:
                left = lefts[index]
                gold_dict[left] = gold_dict[key]

    tmp_dict = {}
    for k in gold_dict.keys():
        tmp_dict[k] = []
    for k, v in gold_dict.items():
        tmp_dict[k] = list(set(v))
    gold_dict = tmp_dict

    return gold_dict

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
time: 11.8 s (started: 2022-09-09 11:23:00 +00:00)


In [70]:
import warnings  
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import tensorflow as tf
from tensorflow.keras import layers

import nltk
from nltk.corpus import stopwords
from nltk import pos_tag

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stops = set(stopwords.words('english'))

from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
!pip install transformers
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
from transformers import RobertaForTokenClassification, RobertaTokenizerFast
from transformers import DistilBertForTokenClassification, DistilBertTokenizerFast
from transformers import AlbertForTokenClassification, AlbertTokenizerFast

!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

# for TPU
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
# import torch_xla
# import torch_xla.core.xla_model as xm

# # for TPU
# device = xm.xla_device()
# torch.set_default_tensor_type('torch.FloatTensor')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
cuda
time: 24.4 s (started: 2022-09

### Test on Generalization

In [71]:
import pandas as pd

tag_dict = {}
tag_dict['Sustainability preoccupations'] = 'I-sus'
tag_dict['Digital transformation'] = 'I-dig'
tag_dict['Change in management'] = 'I-mag'
tag_dict['Innovation activities'] = 'I-inn'
tag_dict['Business Model'] = 'I-bus'
tag_dict['Corporate social responsibility ou CSR'] = 'I-cor'
tag_dict['marco-label'] = 'I-mar'
tag2cat = {v: k for k, v in tag_dict.items()}

labels_to_ids2 = {'O':0, 'I-sus':1, 'I-dig':2, 'I-mag':3, 'I-inn':4, 'I-bus':5, 'I-cor':6, 'I-mar':7}
ids_to_labels2 = {v: k for k, v in labels_to_ids2.items()}

gold_dict = build_gold_dict()

time: 826 ms (started: 2022-09-09 11:23:36 +00:00)


In [72]:
# Stored the semantic scores
def DatasetDistribution_sem_score(datasetdf):
    TAGS2, TEXTS2 = [], []
    for m in range(len(datasetdf)):
        ts = datasetdf['Text_para'].iloc[m].split()
        ks = datasetdf['Keyword'].iloc[m].split(',')
        # deal with the special case
        texts = []
        for t in ts:
            if t == '4.0' or t == 'R' or t == 'D' or t == '&':
                texts.append(t)
            else:
                # text = re.findall(r"([\w']+(?:\S-\S)?[\w'])+", t)
                text = re.findall(r"([\w']+[-]?[\w']+)+", t)
                texts.extend(text)
        # deal with the special case
        keywords = []
        for k in ks:
            if k == 'Maintenance 4.0' or k == 'R & D' or k == 'R & D teams' or k == 'revolution 4.0':
                keywords.append(k.split())
            else:
                keyword = re.findall(r"([\w']+[-]?[\w']+)+", k)
                keywords.append(keyword)

        doc_embedding = model.encode([' '.join(texts)])
        scores = []
        for key in keywords:
            candidate_embeddings = [gold_emb_dict[' '.join(key)]]
            distances = cosine_similarity(doc_embedding, candidate_embeddings)
            scores.append([' '.join(key), distances[0][0]])
        
        TEXTS2.append(texts)
        TAGS2.append(scores)
#         print(texts)
#         print(TAGS2)
    
    print(len(TAGS2), len(TEXTS2))
#     datasetdf['similarity_score'] = TAGS2
#     datasetdf['sentence'] = [' '.join(tx) for tx in TEXTS2]

    return TAGS2, TEXTS2

time: 5.5 ms (started: 2022-09-09 11:23:37 +00:00)


In [73]:
import ast
# Based on Sematic choice
def DatasetBuilder_tr_sem(datasetdf):
    TAGS2, TEXTS2, KWS2 = [], [], []
    for m in range(len(datasetdf)):
        texts = ast.literal_eval(datasetdf['sentences'].iloc[m])
        ks = datasetdf['keywords_0.1'].iloc[m]

        # deal with the special case
        keywords = []
        for k in ks:
            keywords.append(k.split())
            
        tags = ['O'] * len(texts)
        # filter the keywords
#         if len(keywords) > 2:
#             doc_embedding = model.encode([' '.join(texts)])
            
#             candidate_embeddings = [gold_emb_dict[' '.join(key)] for key in keywords]
#             top_n = 2
#             distances = cosine_similarity(doc_embedding, candidate_embeddings)
#             keywords = [keywords[index] for index in distances.argsort()[0][-top_n:]]
        
        # cases: for each of the keyword in each block
        # 1. only one keyword in the block, and no marco lable overlapping
        # 2. more keyword in the block, and no marco label overlapping
        # 3. only one keyword in the block, but there's marco label overlapping
        # 4. more keyword in the block, and maroc label overlapping exists
        for kw in keywords:
            tt = tag_dict[gold_dict[' '.join(kw)][0]]
            length_kw = len(kw)
            item = kw[0]

            if length_kw == 1:
                start = 0
                end = len(texts)
                while True:
                    try: 
                        index = texts.index(item, start, end)
                        tags[index] = tt
                        start = index+1
                    except:
                        break
            
            if length_kw == 2:
                start = 0
                end = len(texts)
                while True:
                    try: 
                        index = texts.index(item, start, end)
                        start = index+1
                        if texts[index+1] == kw[1]:
                            if (tags[index] != 'O' and tags[index] != tt) or (tags[index+1] != 'O' and tags[index+1] != tt):
                                tt = 'I-mar'
                            for i in range(index, index+length_kw):
                                tags[i] = tt
                    except:
                        break

            if length_kw >= 3:
                start = 0
                end = len(texts)
                while True:
                    try: 
                        index = texts.index(item, start, end)
                        start = index+1
                        if texts[index+1] == kw[1] and texts[index+2] == kw[2]:
                            if (tags[index] != 'O' and tags[index] != tt) or (tags[index+1] != 'O' and tags[index+1] != tt) or (tags[index+2] != 'O' and tags[index+2] != tt):
                                tt = 'I-mar'
                            for i in range(index, index+length_kw):
                                tags[i] = tt
                    except:
                        break
        
        KWS2.append(ks)
        TAGS2.append(tags)
        TEXTS2.append(texts)
    
    print(len(TAGS2), len(TEXTS2))
    datasetdf['word_labels2'] = [','.join(bt) for bt in TAGS2]
    datasetdf['sentence'] = [' '.join(tx) for tx in TEXTS2]

    return datasetdf[["sentence", "word_labels2", "keywords_0.1"]].drop_duplicates(subset=["sentence", "word_labels2"]).reset_index(drop=True)
#     data2 = datasetdf.drop_duplicates(subset=["sentence", "word_labels2"])
    
#     data2 = data2[data2['keywords_0.1'].map(len) != 0].reset_index(drop=True)
#     train_indices = []
#     for i in range(len(data2)):
#         kws = data2['keywords_0.1'].iloc[i]
#         for kw in kws:
#             if kw in train_terms:
#                 train_indices.append(i)
#                 break
    
#     return data2[["sentence", "word_labels2"]], train_indices

time: 3.63 ms (started: 2022-09-09 11:23:37 +00:00)


In [74]:
# datasetdf = pd.read_csv("../input/annual-report/dataset_ap_07_18.csv")
# # datasetdf = datasetdf.drop_duplicates(subset=['Company', 'Sector', 'Text_para', 'Nb_company', 'Text_block', 'Catogory', 'Keyword'], keep='first', ignore_index=True)
# datasetdf = datasetdf.drop_duplicates(subset=['Company', 'Year', 'Text_para', 'Text_block', 'Catogory', 'Keyword'], keep='first', ignore_index=True)
# datasetdf.drop(datasetdf.columns[datasetdf.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

# def remove_back_slash_n(text):
#     return text.replace('\n', ' ')
# datasetdf['Text_para'] = datasetdf['Text_para'].apply(remove_back_slash_n)

time: 9.75 ms (started: 2022-09-09 11:23:37 +00:00)


In [75]:
import pickle
with open("../input/dataset-semantic/dataset_semantic_07_22.pkl", "rb") as f:
    datasetdf = pickle.load(f)

time: 908 ms (started: 2022-09-09 11:23:37 +00:00)


In [76]:
# from sklearn.metrics.pairwise import cosine_similarity
# datasetdf = datasetdf[datasetdf['keywords_0.1'].map(len) != 0]
# data2, train_indices = DatasetBuilder_tr_sem(datasetdf)
data2 = DatasetBuilder_tr_sem(datasetdf)

33166 33166
time: 13.4 s (started: 2022-09-09 11:23:38 +00:00)


## There are three ways to divide the dataset

- 75% terms into training + 25% terms into testing -> on web
- Reduction: training is 70% of blocks plus 75% of terms and testing is 30% of blocks plus all terms
- Annotate blocks: missing label, negative labels, ...

In [10]:
# Method 1
keywords = set(kw for kws in datasetdf['keywords_0.1'].values.tolist() for kw in kws)
import random
random.seed(40)

train_terms = random.sample(keywords, int(len(keywords)*0.75))

time: 11.5 ms (started: 2022-09-05 18:57:57 +00:00)


In [10]:
# Method 2
datasetdf = data2
keywords = set(kw for kws in datasetdf['keywords_0.1'].values.tolist() for kw in kws)

keywords2blocks = {kw:[] for kw in keywords}
blocks2keywords = {i:datasetdf['keywords_0.1'].iloc[i] for i in range(len(datasetdf))}

# Update keywords2blocks
for k in keywords2blocks.keys():
    for kb, vb in blocks2keywords.items():
        if k in vb:
            keywords2blocks[k].append(kb)

time: 1.93 s (started: 2022-09-06 17:02:57 +00:00)


In [11]:
np.random.seed(422)
keyword_list, block_list = [], []
for k,v in keywords2blocks.items():
    keyword_list.extend([k]*len(v))
    block_list.extend(v)

reductiondf = pd.DataFrame({'keyword':keyword_list, 'block': block_list})

sample_size = int(len(datasetdf) * 0.2)
sample = reductiondf.groupby('keyword').sample(1, random_state=1)

sample = sample.append(
    reductiondf[~reductiondf.index.isin(sample.index)] # only rows that have not been selected
    .sample(n=sample_size-sample.shape[0]) # sample more rows as needed
).sort_index()

test_indices = set(sample['block'].values.tolist())

# the blocks remained
b2k_remained = {}
for k,v in blocks2keywords.items():
    if k not in test_indices:
        b2k_remained[k] = v

print(f"length of testing set: {len(test_indices)}")

train_size = int(len(test_indices) * (7/3))

# build training set
train_indices = np.random.choice(list(b2k_remained.keys()), size=train_size, replace=False)
train_dict = {}
for k,v in b2k_remained.items():
    if k in train_indices:
        train_dict[k] = v
print(f"length of training set: {len(train_dict)}, {train_size}")
print(f"the proportion of training: {len(train_dict)/(len(test_indices)+len(train_dict))}")
print(f"the proportaion of training terms: {len(set([v for vs in train_dict.values() for v in vs])) / len(keywords):.3f}")

length of testing set: 6096
length of training set: 14224, 14224
the proportion of training: 0.7
the proportaion of training terms: 0.861
time: 345 ms (started: 2022-09-06 17:02:59 +00:00)


In [24]:
# firstly guarantee the proportion of terms
# np.random.seed(422)

# train_terms = np.random.choice(list(keywords), int(len(keywords)*0.8))
# print(f"train terms: {len(train_terms)}")
# block_list = []
# for k,v in keywords2blocks.items():
#     if k in train_terms:
#         block_list.extend(v)
# train_indices = list(set(block_list))
# train_indices = np.random.choice(train_indices, int(len(train_indices)*0.8))

# train_terms_final = []
# for k,v in blocks2keywords.items():
#     if k in train_indices:
#         train_terms_final.extend(v)
# train_terms_final = list(set(train_terms_final))
# print(f"train terms final: {len(train_terms_final)}")

# train_size = len(train_indices)
# test_size = int(train_size * (3/7))
# print(f"train size: {train_size}")
# b2k_remained = {}
# for k,v in blocks2keywords.items():
#     if k not in train_indices:
#         b2k_remained[k] = v

# keyword_list, block_list = [], []
# for k,v in keywords2blocks.items():
#     keyword_list.extend([k]*len(v))
#     block_list.extend(v)

# reductiondf = pd.DataFrame({'keyword':keyword_list, 'block': block_list})

# keyword_list, block_list = [], []
# for i in range(len(reductiondf)):
#     block = reductiondf['block'].iloc[i]
#     keyword = reductiondf['keyword'].iloc[i]
#     if block not in list(train_indices):
#         keyword_list.extend(keyword)
#         block_list.append(block)

train terms: 298
train terms final: 336
train size: 14937
time: 349 ms (started: 2022-09-06 15:07:38 +00:00)


In [10]:
# with open("semantic_dataset_training_07_28.pkl", "wb") as f:
#     pickle.dump(data2, f)

time: 323 µs (started: 2022-09-05 07:25:34 +00:00)


In [77]:
class dataset2(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.sentence[index].strip().split()  
        word_labels = self.data.word_labels2[index].split(",") 

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_split_into_words=True, 
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids2[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

time: 2.99 ms (started: 2022-09-09 11:23:51 +00:00)


In [78]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train2(epoch, training_loader2):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels, tr_probs = [], [], []
    # put model in training mode
    model2.train()
    
    for idx, batch in enumerate(training_loader2):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        results = model2(input_ids=ids, attention_mask=mask, labels=labels)
        loss = results.loss
        tr_logits = results.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model2.num_labels) # shape (batch_size * seq_len, num_labels)
#         flattened_probabilities = F.softmax(active_logits, dim=1) # probabilities
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
#         probabilities = []
#         for i, act in enumerate(active_accuracy):
#             if act:
#                 probabilities.append(flattened_probabilities[i])
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)
#         tr_probs.extend(probabilities)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model2.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer2.zero_grad()
        loss.backward()
        optimizer2.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

    return epoch_loss, tr_accuracy#, tr_labels, tr_preds, tr_probs

time: 2.99 ms (started: 2022-09-09 11:23:51 +00:00)


In [79]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels, eval_probs = [], [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            results = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = results.loss
            eval_logits = results.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            # if idx % 100==0:
            #     loss_step = eval_loss/nb_eval_steps
            #     print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_probabilities = F.softmax(active_logits, dim=1) # probabilities
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            probabilities = []
            for i, act in enumerate(active_accuracy):
                if act:
                    probabilities.append(flattened_probabilities[i])
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            eval_probs.extend(probabilities)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels2[id.item()] for id in eval_labels]
    predictions = [ids_to_labels2[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_loss, eval_accuracy, eval_probs

time: 2.79 ms (started: 2022-09-09 11:23:51 +00:00)


In [80]:
import time

MAX_LEN = 300
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 10
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', add_prefix_space=True)
# tokenizer = AlbertTokenizerFast.from_pretrained('albert-base-v1', add_prefix_space=True)

# K-fold
k_fold = 5
cur_fold = 1
gap = int(len(data2) / k_fold)
indices = [[i*gap, (i+1)*gap if i+1 != k_fold else len(data2)] 
          for i in range(k_fold)]
data3 = data2.sample(frac=1, random_state=200)
test_dataset2 = data3[indices[cur_fold][0]:indices[cur_fold][1]]
train_dataset2 = data3.drop(test_dataset2.index).reset_index(drop=True)
test_dataset2 = test_dataset2.reset_index(drop=True)

# New dividing method
# Method 1
# train_dataset2 = data2.iloc[train_indices]
# test_dataset2 = data2.drop(train_dataset2.index).reset_index(drop=True)
# train_dataset2 = train_dataset2.reset_index(drop=True)
# Method 2
# train_dataset2 = data2.iloc[train_indices].reset_index(drop=True)
# test_dataset2 = data2.iloc[list(test_indices)].reset_index(drop=True)

# train_size = 0.8
# train_dataset2 = data2.sample(frac=train_size,random_state=202)
# test_dataset2 = data2.drop(train_dataset2.index).reset_index(drop=True)
# train_dataset2 = train_dataset2.reset_index(drop=True)

print("TRAIN Dataset: {}".format(train_dataset2.shape))
print("TEST Dataset: {}".format(test_dataset2.shape))

training_set2 = dataset2(train_dataset2, tokenizer, MAX_LEN)
testing_set2 = dataset2(test_dataset2, tokenizer, MAX_LEN)

train_params2 = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params2 = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
#                 'shuffle': True,
                'num_workers': 0
                }

training_loader2 = DataLoader(training_set2, **train_params2)
testing_loader2 = DataLoader(testing_set2, **test_params2)

# model2 = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(labels_to_ids2))
# model2 = BertForTokenClassification.from_pretrained('bert-large-cased', num_labels=len(labels_to_ids2))
# model2 = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(labels_to_ids2))
model2 = RobertaForTokenClassification.from_pretrained('roberta-base', num_labels=len(labels_to_ids2))
# model2 = AlbertForTokenClassification.from_pretrained('albert-base-v1', num_labels=len(labels_to_ids2))
model2.to(device)

optimizer2 = torch.optim.Adam(params=model2.parameters(), lr=LEARNING_RATE)

trLosslist, trAcclist, evalLosslist, evalAcclist, timelist = [], [], [], [], []
label_pred_dict = {}
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    tmp_dict = {}
    start = time.time()
    epoch_loss, tr_accuracy = train2(epoch, training_loader2)
    labels, predictions, eval_loss, eval_accuracy, eval_probs = valid(model2, testing_loader2)
    epoch_time = time.time() - start
    trLosslist.append(epoch_loss)
    trAcclist.append(tr_accuracy)
    evalLosslist.append(eval_loss)
    evalAcclist.append(eval_accuracy)
    timelist.append(epoch_time)
#     tmp_dict["predictions"] = predictions
#     tmp_dict["labels"] = labels
#     tmp_dict["probabilities"] = eval_probs
#     label_pred_dict[epoch] = tmp_dict

resultdf = pd.DataFrame({"Epoch": list(range(1, EPOCHS+1)),
                         "Train_loss": trLosslist,
                         "Eval_loss": evalLosslist,
                         "Train_Acc": trAcclist,
                         "Eval_Acc": evalAcclist,
                        "Time": timelist})
resultdf    

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

TRAIN Dataset: (26362, 3)
TEST Dataset: (6590, 3)


Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able

Training epoch: 1
Training loss per 100 training steps: 2.112478494644165
Training loss per 100 training steps: 0.26645716170937117
Training loss per 100 training steps: 0.1907427158132566
Training loss per 100 training steps: 0.15235028500077732
Training loss per 100 training steps: 0.1269413362305955
Training loss per 100 training steps: 0.10919168517606968
Training loss per 100 training steps: 0.09581460379538267
Training loss per 100 training steps: 0.08616914597410343
Training loss per 100 training steps: 0.0788365905879463
Training loss per 100 training steps: 0.07276395568494932
Training loss per 100 training steps: 0.06737161724147601
Training loss per 100 training steps: 0.06324905563147144
Training loss per 100 training steps: 0.059688685438324306
Training loss per 100 training steps: 0.056565921461309156
Training loss per 100 training steps: 0.05388126501969416
Training loss per 100 training steps: 0.05174717426166338
Training loss per 100 training steps: 0.04965943842493684

KeyboardInterrupt: 

time: 2h 14min 8s (started: 2022-09-09 11:24:57 +00:00)


In [81]:
# NEW ONE newly generated words
def lengthOfTokens(pair):
        return len(pair.split())

def tags_to_keywords(sample, words):
    indices = [i for i, l in enumerate(sample) if l != 'O']
    keywords, key_cats = [], []
    for j, id in enumerate(indices):
        if j == 0:
            start = end = id
            continue
        if j == len(indices):
            pos = pos_tag(words[start:end+1])
            if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
                keywords.append(' '.join(words[start:end+1]))
                key_cats.append((' '.join(words[start:end+1]), sample[start:end+1]))
            continue
        if end+1 == id:
            end = id
        else:
            pos = pos_tag(words[start:end+1])
            if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
                keywords.append(' '.join(words[start:end+1]))
                key_cats.append((' '.join(words[start:end+1]), sample[start:end+1]))
            start = end = id
    return list(set(keywords)), key_cats

def countKeywords(test_dataset, model):
    kws_pairs = []
    Preds, Preds_cats, Lbs, Lbs_cats = [], [], [], []
    for tmp_num in range(len(test_dataset)):
        sentence = test_dataset["sentence"].iloc[tmp_num]

        inputs = tokenizer(sentence.split(),
                            is_split_into_words=True, 
                            return_offsets_mapping=True, 
                            padding='max_length', 
                            truncation=True, 
                            max_length=MAX_LEN,
                            return_tensors="pt")

        # move to gpu
        ids = inputs["input_ids"].to(device)
        mask = inputs["attention_mask"].to(device)
        # forward pass
        outputs = model(ids, attention_mask=mask)
        logits = outputs.logits

        active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

        tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
        token_predictions = [ids_to_labels2[i] for i in flattened_predictions.cpu().numpy()]
        wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

        prediction = []
        for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
          #only predictions on first word pieces are important
          if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
          else:
            continue
        
        # predictions
        preds, preds_cats = tags_to_keywords(prediction, sentence.split())
        lbs, lbs_cats = tags_to_keywords(test_dataset["word_labels2"].iloc[tmp_num].split(','), sentence.split())

        Preds.append(preds)
        Preds_cats.append(preds_cats)
        Lbs.append(lbs)
        Lbs_cats.append(lbs_cats)
        
    return Preds, Preds_cats, Lbs, Lbs_cats

time: 5.78 ms (started: 2022-09-09 13:39:12 +00:00)


In [82]:
Preds, Preds_cats, Lbs, Lbs_cats = countKeywords(test_dataset2, model2)

time: 2min 13s (started: 2022-09-09 13:39:13 +00:00)


In [83]:
pred_lable_keywords = {}
pred_lable_keywords['tag_preds'] = predictions
pred_lable_keywords['tag_labels'] = labels
pred_lable_keywords['predicitons'] = Preds
pred_lable_keywords['preds_cats'] = Preds_cats
pred_lable_keywords['labels'] = Lbs
pred_lable_keywords['labels_cats'] = Lbs_cats

time: 833 µs (started: 2022-09-09 13:41:26 +00:00)


In [84]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def get_recall(Labels, Predictions, ignore=True):
    recalls = []
    for i, lbs in enumerate(Labels):
        preds = Predictions[i]
        if not lbs:
            if not ignore:
                recalls.append(1.0 if not preds else 0.0)
            continue
        recalls.append(len(intersection(preds, lbs)) / len(lbs))
    return np.mean(recalls)

def get_precision(Labels, Predictions, ignore=True):
    precisions = []
    for i, preds in enumerate(Predictions):
        lbs = Labels[i]
        if not preds:
            if not ignore:
                precisions.append(1.0 if not preds else 0.0)
            continue
        precisions.append(len(intersection(preds, lbs)) / len(preds))
    return np.mean(precisions)

def get_f1score(Labels, Predictions, ignore=True):
    precision = get_precision(Labels, Predictions, ignore)
    recall = get_recall(Labels, Predictions, ignore)
    return 2 * precision * recall / (precision + recall)

time: 2.56 ms (started: 2022-09-09 13:41:26 +00:00)


In [85]:
predictions = pred_lable_keywords['predicitons']
labels = pred_lable_keywords['labels']

print(get_recall(labels, predictions, False))
print(get_precision(labels, predictions, False))
print(get_f1score(labels, predictions, False))
print(get_recall(labels, predictions))
print(get_precision(labels, predictions))
print(get_f1score(labels, predictions))

0.9264653909570448
0.9651591516728086
0.9454165249380461
0.842757306888265
0.8856567776512996
0.8636746569437744
time: 38.4 ms (started: 2022-09-09 13:41:26 +00:00)


In [18]:
# import pickle

# with open(f"./albert-noblock-label-pred-keywords-kfold{cur_fold}.pkl", "wb") as f:
#     pickle.dump(pred_lable_keywords, f)

# resultdf.to_csv(f"./albert-noblock-loss-acc-kfold{cur_fold}.csv")

import pickle

with open(f"./albert-label-pred-keywords-kfold{cur_fold}.pkl", "wb") as f:
    pickle.dump(pred_lable_keywords, f)

resultdf.to_csv(f"./albert-loss-acc-kfold{cur_fold}.csv")

time: 66 ms (started: 2022-09-05 08:37:02 +00:00)


In [19]:
model2.save_pretrained(f"./albert_epoch4-kfold{cur_fold}")

time: 76.4 ms (started: 2022-09-05 08:37:02 +00:00)


# BERT + CRF

In [52]:
!pip install pytorch_pretrained_bert==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.3/114.3 kB 571.9 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: pytorch_pretrained_bert
    Found existing installation: pytorch-pretrained-bert 0.6.2
    Uninstalling pytorch-pretrained-bert-0.6.2:
      Successfully uninstalled pytorch-pretrained-bert-0.6.2
time: 28.5 s (started: 2022-09-09 10:46:34 +00:00)


In [54]:
import sys
import os
import time
import importlib
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim

from torch.utils.data.distributed import DistributedSampler
from torch.utils import data

from tqdm import tqdm, trange
import collections

from pytorch_pretrained_bert.modeling import BertModel, BertForTokenClassification, BertLayerNorm
import pickle
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule
from pytorch_pretrained_bert.tokenization import BertTokenizer

time: 1.6 ms (started: 2022-09-09 10:47:20 +00:00)


In [44]:
def log_sum_exp_1vec(vec):  # shape(1,m)
    max_score = vec[0, np.argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

def log_sum_exp_mat(log_M, axis=-1):  # shape(n,m)
    return torch.max(log_M, axis)[0]+torch.log(torch.exp(log_M-torch.max(log_M, axis)[0][:, None]).sum(axis))

def log_sum_exp_batch(log_Tensor, axis=-1): # shape (batch_size,n,m)
    return torch.max(log_Tensor, axis)[0]+torch.log(torch.exp(log_Tensor-torch.max(log_Tensor, axis)[0].view(log_Tensor.shape[0],-1,1)).sum(axis))


class BERT_CRF_NER(nn.Module):

    def __init__(self, bert_model, start_label_id, stop_label_id, num_labels, max_seq_length, batch_size, device):
        super(BERT_CRF_NER, self).__init__()
        self.hidden_size = 768
        self.start_label_id = start_label_id
        self.stop_label_id = stop_label_id
        self.num_labels = num_labels
        # self.max_seq_length = max_seq_length
        self.batch_size = batch_size
        self.device=device

        # use pretrainded BertModel
        self.bert = bert_model
        self.dropout = torch.nn.Dropout(0.2)
        # Maps the output of the bert into label space.
        self.hidden2label = nn.Linear(self.hidden_size, self.num_labels)

        # Matrix of transition parameters.  Entry i,j is the score of transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.num_labels, self.num_labels))

        # These two statements enforce the constraint that we never transfer *to* the start tag(or label),
        # and we never transfer *from* the stop label (the model would probably learn this anyway,
        # so this enforcement is likely unimportant)
        self.transitions.data[start_label_id, :] = -10000
        self.transitions.data[:, stop_label_id] = -10000

        nn.init.xavier_uniform_(self.hidden2label.weight)
        nn.init.constant_(self.hidden2label.bias, 0.0)
        # self.apply(self.init_bert_weights)

    def init_bert_weights(self, module):
        """ Initialize the weights.
        """
        if isinstance(module, (nn.Linear, nn.Embedding)):
            # Slightly different from the TF version which uses truncated_normal for initialization
            # cf https://github.com/pytorch/pytorch/pull/5617
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
        elif isinstance(module, BertLayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def _forward_alg(self, feats):
        '''
        this also called alpha-recursion or forward recursion, to calculate log_prob of all barX
        '''

        # T = self.max_seq_length
        T = feats.shape[1]
        batch_size = feats.shape[0]

        # alpha_recursion,forward, alpha(zt)=p(zt,bar_x_1:t)
        log_alpha = torch.Tensor(batch_size, 1, self.num_labels).fill_(-10000.).to(self.device)
        # normal_alpha_0 : alpha[0]=Ot[0]*self.PIs
        # self.start_label has all of the score. it is log,0 is p=1
        log_alpha[:, 0, self.start_label_id] = 0

        # feats: sentances -> word embedding -> lstm -> MLP -> feats
        # feats is the probability of emission, feat.shape=(1,tag_size)
        for t in range(1, T):
            log_alpha = (log_sum_exp_batch(self.transitions + log_alpha, axis=-1) + feats[:, t]).unsqueeze(1)

        # log_prob of all barX
        log_prob_all_barX = log_sum_exp_batch(log_alpha)
        return log_prob_all_barX

    def _get_bert_features(self, input_ids, segment_ids, input_mask):
        '''
        sentances -> word embedding -> lstm -> MLP -> feats
        '''
        bert_seq_out, _ = self.bert(input_ids, token_type_ids=segment_ids, attention_mask=input_mask, output_all_encoded_layers=False)
        bert_seq_out = self.dropout(bert_seq_out)
        bert_feats = self.hidden2label(bert_seq_out)
        return bert_feats

    def _score_sentence(self, feats, label_ids):
        '''
        Gives the score of a provided label sequence
        p(X=w1:t,Zt=tag1:t)=...p(Zt=tag_t|Zt-1=tag_t-1)p(xt|Zt=tag_t)...
        '''

        # T = self.max_seq_length
        T = feats.shape[1]
        batch_size = feats.shape[0]

        batch_transitions = self.transitions.expand(batch_size,self.num_labels,self.num_labels)
        batch_transitions = batch_transitions.flatten(1)

        score = torch.zeros((feats.shape[0],1)).to(device)
        # the 0th node is start_label->start_word,the probability of them=1. so t begin with 1.
        for t in range(1, T):
            score = score + \
                batch_transitions.gather(-1, (label_ids[:, t]*self.num_labels+label_ids[:, t-1]).view(-1,1)) \
                    + feats[:, t].gather(-1, label_ids[:, t].view(-1,1)).view(-1,1)
        return score

    def _viterbi_decode(self, feats):
        '''
        Max-Product Algorithm or viterbi algorithm, argmax(p(z_0:t|x_0:t))
        '''

        # T = self.max_seq_length
        T = feats.shape[1]
        batch_size = feats.shape[0]

        # batch_transitions=self.transitions.expand(batch_size,self.num_labels,self.num_labels)

        log_delta = torch.Tensor(batch_size, 1, self.num_labels).fill_(-10000.).to(self.device)
        log_delta[:, 0, self.start_label_id] = 0

        # psi is for the vaule of the last latent that make P(this_latent) maximum.
        psi = torch.zeros((batch_size, T, self.num_labels), dtype=torch.long).to(self.device)  # psi[0]=0000 useless
        for t in range(1, T):
            # delta[t][k]=max_z1:t-1( p(x1,x2,...,xt,z1,z2,...,zt-1,zt=k|theta) )
            # delta[t] is the max prob of the path from  z_t-1 to z_t[k]
            log_delta, psi[:, t] = torch.max(self.transitions + log_delta, -1)
            # psi[t][k]=argmax_z1:t-1( p(x1,x2,...,xt,z1,z2,...,zt-1,zt=k|theta) )
            # psi[t][k] is the path choosed from z_t-1 to z_t[k],the value is the z_state(is k) index of z_t-1
            log_delta = (log_delta + feats[:, t]).unsqueeze(1)

        # trace back
        path = torch.zeros((batch_size, T), dtype=torch.long).to(self.device)

        # max p(z1:t,all_x|theta)
        max_logLL_allz_allx, path[:, -1] = torch.max(log_delta.squeeze(), -1)

        for t in range(T-2, -1, -1):
            # choose the state of z_t according the state choosed of z_t+1.
            path[:, t] = psi[:, t+1].gather(-1,path[:, t+1].view(-1,1)).squeeze()

        return max_logLL_allz_allx, path

    def neg_log_likelihood(self, input_ids, segment_ids, input_mask, label_ids):
        bert_feats = self._get_bert_features(input_ids, segment_ids, input_mask)
        forward_score = self._forward_alg(bert_feats)
        # p(X=w1:t,Zt=tag1:t)=...p(Zt=tag_t|Zt-1=tag_t-1)p(xt|Zt=tag_t)...
        gold_score = self._score_sentence(bert_feats, label_ids)
        # - log[ p(X=w1:t,Zt=tag1:t)/p(X=w1:t) ] = - log[ p(Zt=tag1:t|X=w1:t) ]
        return torch.mean(forward_score - gold_score)

    # this forward is just for predict, not for train
    # dont confuse this with _forward_alg above.
    def forward(self, input_ids, segment_ids, input_mask):
        # Get the emission scores from the BiLSTM
        bert_feats = self._get_bert_features(input_ids, segment_ids, input_mask)

        # Find the best path, given the features.
        score, label_seq_ids = self._viterbi_decode(bert_feats)
        return score, label_seq_ids

time: 14.4 ms (started: 2022-09-09 10:27:54 +00:00)


In [48]:
import time

MAX_LEN = 300
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 4
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
# tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', add_prefix_space=True)
# tokenizer = AlbertTokenizerFast.from_pretrained('albert-base-v1', add_prefix_space=True)

# K-fold
# k_fold = 5
# cur_fold = 4
# gap = int(len(data2) / k_fold)
# indices = [[i*gap, (i+1)*gap if i+1 != k_fold else len(data2)] 
#           for i in range(k_fold)]
# data3 = data2.sample(frac=1, random_state=200)
# test_dataset2 = data3[indices[cur_fold][0]:indices[cur_fold][1]]
# train_dataset2 = data3.drop(test_dataset2.index).reset_index(drop=True)
# test_dataset2 = test_dataset2.reset_index(drop=True)

# New dividing method
# Method 1
# train_dataset2 = data2.iloc[train_indices]
# test_dataset2 = data2.drop(train_dataset2.index).reset_index(drop=True)
# train_dataset2 = train_dataset2.reset_index(drop=True)
# Method 2
# train_dataset2 = data2.iloc[train_indices].reset_index(drop=True)
# test_dataset2 = data2.iloc[list(test_indices)].reset_index(drop=True)

train_size = 0.8
train_dataset2 = data2.sample(frac=train_size,random_state=202)
test_dataset2 = data2.drop(train_dataset2.index).reset_index(drop=True)
train_dataset2 = train_dataset2.reset_index(drop=True)

print("TRAIN Dataset: {}".format(train_dataset2.shape))
print("TEST Dataset: {}".format(test_dataset2.shape))

training_set2 = dataset2(train_dataset2, tokenizer, MAX_LEN)
testing_set2 = dataset2(test_dataset2, tokenizer, MAX_LEN)

train_params2 = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params2 = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader2 = DataLoader(training_set2, **train_params2)
testing_loader2 = DataLoader(testing_set2, **test_params2)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

TRAIN Dataset: (26362, 3)
TEST Dataset: (6590, 3)
time: 5.82 s (started: 2022-09-09 10:40:50 +00:00)


In [64]:
counter = 0
for batch in training_loader2:
    print(batch['input_ids'])
    print(batch['labels'])
    counter += 1
    if counter > 5:
        break

tensor([[  101,  5514,  1329,  ...,     0,     0,     0],
        [  101,  1857,  1203,  ...,     0,     0,     0],
        [  101,  1109,  6986,  ...,  4679, 11478,   102],
        [  101, 24819, 12880,  ...,  1559,  8653,   102]])
tensor([[-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0, -100,  ..., -100,    0, -100]])
tensor([[ 101, 1130, 1103,  ...,    0,    0,    0],
        [ 101, 1130, 1142,  ...,    0,    0,    0],
        [ 101, 8094, 1112,  ...,    0,    0,    0],
        [ 101, 4602, 1106,  ...,    0,    0,    0]])
tensor([[-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100]])
tensor([[  101,  1130, 10351,  ...,     0,     0,     0],
        [  101,  1130,  1901,  ...,     0,     0,     0],
        [  101, 22439,  

In [66]:
start_label_id = -100
stop_label_id = -100
bert_model = BertModel.from_pretrained('bert-base-cased')
model2 = BERT_CRF_NER(bert_model, start_label_id, stop_label_id, len(labels_to_ids2), MAX_LEN, batch_size, device)
model2.to(device)

optimizer2 = torch.optim.Adam(params=model2.parameters(), lr=LEARNING_RATE)

trLosslist, trAcclist, evalLosslist, evalAcclist, timelist = [], [], [], [], []
label_pred_dict = {}
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    tmp_dict = {}
    start = time.time()
    epoch_loss, tr_accuracy = train2(epoch, training_loader2)
    labels, predictions, eval_loss, eval_accuracy, eval_probs = valid(model2, testing_loader2)
    epoch_time = time.time() - start
    trLosslist.append(epoch_loss)
    trAcclist.append(tr_accuracy)
    evalLosslist.append(eval_loss)
    evalAcclist.append(eval_accuracy)
    timelist.append(epoch_time)

resultdf = pd.DataFrame({"Epoch": list(range(1, EPOCHS+1)),
                         "Train_loss": trLosslist,
                         "Eval_loss": evalLosslist,
                         "Train_Acc": trAcclist,
                         "Eval_Acc": evalAcclist,
                        "Time": timelist})
resultdf  

IndexError: index -100 is out of bounds for dimension 0 with size 8

time: 8.89 s (started: 2022-09-09 11:10:38 +00:00)


### Version 1 only keywords

In [34]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def get_recall(Labels, Predictions):
    recalls = []
    for i, lbs in enumerate(Labels):
        preds = Predictions[i]
        recalls.append(len(intersection(preds, lbs)) / len(lbs))
    return np.mean(recalls)

def get_precision(Labels, Predictions):
    precisions = []
    for i, preds in enumerate(Predictions):
        lbs = Labels[i]
        precisions.append(len(intersection(preds, lbs)) / len(preds))
    return np.mean(precisions)

def get_f1score(Labels, Predictions):
    precision = get_precision(Labels, Predictions)
    recall = get_recall(Labels, Predictions)
    return [(precision[i][0], 
             np.nan_to_num(2 * precision[i][1] * recall[i][1] / (precision[i][1] + recall[i][1])))
            for i in range(len(matrix)-1)]

[('research and development', ['I-inn', 'I-inn', 'I-bus']),
 ('artificial intelligence research project',
  ['I-dig', 'I-dig', 'I-inn', 'I-inn']),
 ('research centers', ['I-inn', 'I-dig']),
 ('product portfolio product development',
  ['I-bus', 'I-bus', 'I-inn', 'I-inn']),
 ('research and development', ['I-inn', 'I-inn', 'I-bus']),
 ('research and development', ['I-inn', 'I-inn', 'I-bus'])]

time: 8.97 ms (started: 2022-08-09 14:32:00 +00:00)


In [14]:
# OLD ONE newly generated words
def lengthOfTokens(pair):
        return len(pair.split())

def countKeywords(test_dataset, model):
    kws_pairs = []
    
    for tmp_num in range(len(test_dataset)):
        sentence = test_dataset["sentence"].iloc[tmp_num]

        inputs = tokenizer(sentence.split(),
                            is_split_into_words=True, 
                            return_offsets_mapping=True, 
                            padding='max_length', 
                            truncation=True, 
                            max_length=MAX_LEN,
                            return_tensors="pt")

        # move to gpu
        ids = inputs["input_ids"].to(device)
        mask = inputs["attention_mask"].to(device)
        # forward pass
        outputs = model(ids, attention_mask=mask)
        logits = outputs.logits

        active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

        tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
        token_predictions = [ids_to_labels2[i] for i in flattened_predictions.cpu().numpy()]
        wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

        prediction = []
        for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
          #only predictions on first word pieces are important
          if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
          else:
            continue

        for i in zip(sentence.split(), prediction, test_dataset["word_labels2"].iloc[tmp_num].split(',')):
            kws_pairs.append(i)

    predict_kws = []
    tmp_wd = []
    tmp_pred, tmp_gold = '', ''
    for word, pred, gold in kws_pairs:
        if pred != 'O':
            if len(tmp_wd) == 0: 
                tmp_pred = pred
            else:
                if tmp_pred != pred:
                    predict_kws.append((' '.join(tmp_wd), tmp_pred))
                    tmp_wd = []
                    tmp_pred = pred
            tmp_wd.append(word)
        else:
            if len(tmp_wd) > 0:
                predict_kws.append((' '.join(tmp_wd), tmp_pred))
                tmp_wd = []
                tmp_pred = pred

    # check whether there are new keywords
    gold_dict = build_gold_dict()
    new_kws, kw_tags = [], []
    gold_lists = list(gold_dict.keys())
    gold_lists = [g.lower() for g in gold_lists]
    for word, label in predict_kws:
        if not word.lower() in gold_lists:
            new_kws.append(word.lower())
            kw_tags.append(tag2cat[label])
    new_keywords = list(zip(new_kws, kw_tags))

    new_kws_dict, new_kws_pos = {}, []
    for kw, cat in new_keywords:
        pos = pos_tag(kw.split())
        if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
            key = ' '.join([p for p, t in pos])
            tag = [t for p, t in pos]
            new_kws_pos.append(key)
            if not key in new_kws_dict.keys():
                new_kws_dict[key] = (tag, cat)

    new_kws = list(new_kws_dict.keys())
    cats = [value[1] for value in new_kws_dict.values()]

    df = pd.DataFrame({"keyword": new_kws, "category": cats})

    s = df['keyword'].apply(lengthOfTokens)
    s.sort_values()

    predict_kws = Counter(predict_kws)
    new_kws_pos = Counter(new_kws_pos)

    predict_kws = sorted(predict_kws.items(), key=lambda pair: pair[1], reverse=True)
    new_kws_pos = sorted(new_kws_pos.items(), key=lambda pair: pair[1], reverse=True)

    return predict_kws, new_kws_pos, df

time: 5.12 ms (started: 2022-07-29 14:50:38 +00:00)


In [15]:
new_kws_dict = {}
new_kw_df = pd.DataFrame({})
for i in range(4):
    labels = label_pred_dict[i]["labels"]
    predictions = label_pred_dict[i]["predictions"]

    predict_kws, new_kws_pos, tmp_kws_df = countKeywords(test_dataset2, model2)
    tmp_kws_df['epoch'] = [i+1]*len(tmp_kws_df)

    new_kws_dict['predicted_keywords_epoch_' + str(i+1)] = predict_kws
    new_kws_dict['new_keywords_epoch_' + str(i+1)] = new_kws_pos

    new_kw_df = pd.concat([new_kw_df, tmp_kws_df])
    new_kw_df.reset_index(drop=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  


time: 4min 43s (started: 2022-07-29 14:50:40 +00:00)


In [16]:
for i in range(4):
    predict_kws = new_kws_dict['new_keywords_epoch_' + str(i+1)]
    print(f"In Epoch {i+1}:")
    print(predict_kws)

In Epoch 1:
[('development', 52), ('research and development costs', 22), ('software development costs', 11), ('product development costs', 11), ('property', 10), ('programs', 8), ('energy', 7), ('data', 7), ('research and development services', 6), ('storage', 5), ('costs', 5), ('business', 5), ('product', 5), ('research', 5), ('solutions', 4), ('design', 4), ('chain', 4), ('carbon', 4), ('new industry', 4), ('artificial intelligence machine learning', 4), ('personal data data', 4), ('efforts', 3), ('co2', 3), ('product development projects', 3), ('product development efforts', 3), ('innovation', 3), ('technology', 3), ('cutting-edge', 3), ('design kit', 3), ('customer', 2), ('transformation', 2), ('climate', 2), ('efficiency', 2), ('science', 2), ('development problems', 2), ('quality', 2), ('iot', 2), ('innovative skills', 2), ('personal data protection', 2), ('research and development and commercialization', 2), ('innovation programs', 2), ('process', 2), ('center', 2), ('system de

In [34]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

def get_distribution(inputs, name):
    true_res = {}
    for k, v in tag2cat.items():
        indexlist = get_index_positions(inputs, k)
        left, right = 0, 0
        new_labels = []
        for i in indexlist:
            if left == 0:
                left = i
                right = i
                continue
            if right+1 == i:
                right = i
            else:
                new_labels.append((left, right))
                left = i
                right = i
        true_res[v] = new_labels
    print(f"The distribution of {name} is:")
    for k,v in true_res.items():
        print(f"There are {len(v)} occurances in {k}")
    return true_res

def get_confusion_matrix(true_labels, true_predictions):
    # strict mode
    tp_dict = {}
    for true_k, true_v in true_labels.items():
        true_total = len(true_v)
        tmp_dict = {}
        tmp_total = 0
        for pred_k, pred_v in true_predictions.items():
            num = 0
            for v in pred_v:
                if v in true_v:
                    num += 1
            tmp_dict[pred_k] = num
            tmp_total += num
        tmp_dict["O"] = true_total - tmp_total
        tp_dict[true_k] = tmp_dict
    tp_dict
    tmp_dict = {}
    for pred_k, pred_v in true_predictions.items():
        true_total = len(pred_v)
        num = 0
        for tp_k, tp_v in tp_dict.items():
            num += tp_v[pred_k]
        tmp_dict[pred_k] = true_total - num
    tmp_dict["O"] = 0
    tp_dict["O"] = tmp_dict
    return pd.DataFrame(tp_dict).T

time: 2.3 ms (started: 2022-08-02 11:10:59 +00:00)


In [18]:
for i in range(4):
    print(f"In Epoch {i+1}:")
    labels = label_pred_dict[i]["labels"]
    predictions = label_pred_dict[i]["predictions"]

    true_labels = get_distribution(labels, "labels")
    true_predictions = get_distribution(predictions, "predictions")
    df = get_confusion_matrix(true_labels, true_predictions)
    display(df)

In Epoch 1:
The distribution of labels is:
There are 1525 occurances in Sustainability preoccupations
There are 1112 occurances in Digital transformation
There are 200 occurances in Change in management
There are 5219 occurances in Innovation activities
There are 1293 occurances in Business Model
There are 35 occurances in Corporate social responsibility ou CSR
There are 1 occurances in marco-label
The distribution of predictions is:
There are 1737 occurances in Sustainability preoccupations
There are 1146 occurances in Digital transformation
There are 221 occurances in Change in management
There are 5274 occurances in Innovation activities
There are 1392 occurances in Business Model
There are 0 occurances in Corporate social responsibility ou CSR
There are 0 occurances in marco-label


,Sustainability preoccupations,Digital transformation,Change in management,Innovation activities,Business Model,Corporate social responsibility ou CSR,marco-label,O
Sustainability preoccupations,1373,0,0,0,0,0,0,152
Digital transformation,0,931,0,0,0,0,0,181
Change in management,0,0,164,1,0,0,0,35
Innovation activities,0,0,3,4237,1,0,0,978
Business Model,0,0,0,0,1252,0,0,41
Corporate social responsibility ou CSR,0,0,0,0,0,0,0,35
marco-label,0,0,0,0,1,0,0,0
O,364,215,54,1036,138,0,0,0


In Epoch 2:
The distribution of labels is:
There are 1525 occurances in Sustainability preoccupations
There are 1112 occurances in Digital transformation
There are 200 occurances in Change in management
There are 5219 occurances in Innovation activities
There are 1293 occurances in Business Model
There are 35 occurances in Corporate social responsibility ou CSR
There are 1 occurances in marco-label
The distribution of predictions is:
There are 1890 occurances in Sustainability preoccupations
There are 1341 occurances in Digital transformation
There are 209 occurances in Change in management
There are 5800 occurances in Innovation activities
There are 1349 occurances in Business Model
There are 22 occurances in Corporate social responsibility ou CSR
There are 0 occurances in marco-label


,Sustainability preoccupations,Digital transformation,Change in management,Innovation activities,Business Model,Corporate social responsibility ou CSR,marco-label,O
Sustainability preoccupations,1447,0,0,0,0,0,0,78
Digital transformation,0,1049,0,0,0,0,0,63
Change in management,0,0,172,0,0,0,0,28
Innovation activities,0,0,1,4736,0,0,0,482
Business Model,0,0,0,0,1223,0,0,70
Corporate social responsibility ou CSR,0,0,0,0,0,15,0,20
marco-label,0,0,0,0,1,0,0,0
O,443,292,36,1064,125,7,0,0


In Epoch 3:
The distribution of labels is:
There are 1525 occurances in Sustainability preoccupations
There are 1112 occurances in Digital transformation
There are 200 occurances in Change in management
There are 5219 occurances in Innovation activities
There are 1293 occurances in Business Model
There are 35 occurances in Corporate social responsibility ou CSR
There are 1 occurances in marco-label
The distribution of predictions is:
There are 1685 occurances in Sustainability preoccupations
There are 1346 occurances in Digital transformation
There are 276 occurances in Change in management
There are 5444 occurances in Innovation activities
There are 1320 occurances in Business Model
There are 26 occurances in Corporate social responsibility ou CSR
There are 0 occurances in marco-label


,Sustainability preoccupations,Digital transformation,Change in management,Innovation activities,Business Model,Corporate social responsibility ou CSR,marco-label,O
Sustainability preoccupations,1390,0,0,0,0,0,0,135
Digital transformation,0,1054,0,0,0,0,0,58
Change in management,0,0,193,0,0,0,0,7
Innovation activities,0,0,3,4632,0,0,0,584
Business Model,0,0,0,0,1226,0,0,67
Corporate social responsibility ou CSR,0,0,0,0,0,14,0,21
marco-label,0,0,0,0,0,0,0,1
O,295,292,80,812,94,12,0,0


In Epoch 4:
The distribution of labels is:
There are 1525 occurances in Sustainability preoccupations
There are 1112 occurances in Digital transformation
There are 200 occurances in Change in management
There are 5219 occurances in Innovation activities
There are 1293 occurances in Business Model
There are 35 occurances in Corporate social responsibility ou CSR
There are 1 occurances in marco-label
The distribution of predictions is:
There are 1624 occurances in Sustainability preoccupations
There are 1247 occurances in Digital transformation
There are 236 occurances in Change in management
There are 5877 occurances in Innovation activities
There are 1363 occurances in Business Model
There are 5 occurances in Corporate social responsibility ou CSR
There are 0 occurances in marco-label


,Sustainability preoccupations,Digital transformation,Change in management,Innovation activities,Business Model,Corporate social responsibility ou CSR,marco-label,O
Sustainability preoccupations,1401,0,0,0,0,0,0,124
Digital transformation,0,1031,0,0,0,0,0,81
Change in management,0,0,192,0,0,0,0,8
Innovation activities,0,0,1,4892,0,0,0,326
Business Model,0,0,0,0,1253,0,0,40
Corporate social responsibility ou CSR,0,0,0,0,0,3,0,32
marco-label,0,0,0,0,0,0,0,1
O,223,216,43,985,110,2,0,0


time: 6.74 s (started: 2022-07-29 14:55:23 +00:00)


In [1]:
# ./bert-base-cased-keywords-epoch4-9vs1.pkl
# ./bert-base-cased-new-keywords-epoch4-9vs1.csv

# ./bert-base-cased-keywords-epoch4-dropout5.pkl
# ./bert-base-cased-new-keywords-epoch4-dropout5.csv
# ./bert-base-cased-loss-acc-epoch4-dropout5.csv
# ./model_epoch4_dropout5

# Best
# ./bert-base-cased-keywords-epoch4-dropout2.pkl
# ./bert-base-cased-new-keywords-epoch4-dropout2.csv
# ./bert-base-cased-loss-acc-epoch4-dropout2.csv
# ./model_epoch4_dropout2

# semantic
# ./bert-base-cased-keywords-epoch4-semantic2.pkl
# ./bert-base-cased-new-keywords-epoch4-semantic2.csv
# ./bert-base-cased-loss-acc-epoch4-semantic2.csv
# ./model_epoch4_semantic2

import pickle

# store
# label_pred_dict

with open("./distilbert-label-pred-dict.pkl", "wb") as f:
    pickle.dump(label_pred_dict, f)
    
with open("./distilbert-keywords-epoch4-semantic2.pkl", "wb") as f:
    pickle.dump(new_kws_dict, f)

new_kw_df.to_csv("./distilbert-new-keywords-epoch4-semantic2.csv")  

resultdf.to_csv("./distilbert-loss-acc-epoch4-semantic2.csv")

model2.save_pretrained("./distilbert_epoch4_semantic2")

NameError: name 'label_pred_dict' is not defined

In [4]:
df = pd.read_csv("/kaggle/working/bert-base-cased-new-keywords-epoch4-9vs1.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/bert-base-cased-new-keywords-epoch4-9vs1.csv'

time: 40.4 ms (started: 2022-07-21 10:02:01 +00:00)


### Test on Generalizability

### Test on Annual Report

### CNN - from notebook

In [12]:
import functools
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda
time: 84.4 ms (started: 2022-08-11 20:50:14 +00:00)


In [13]:
seed = 0

torch.manual_seed(seed)

time: 8.02 ms (started: 2022-08-11 20:50:14 +00:00)


In [14]:
from allennlp.modules import ConditionalRandomField

time: 542 ms (started: 2022-08-11 20:50:14 +00:00)


In [15]:
class CNN_n(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, dropout_rate, pad_index, crf=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        self.conv1=torch.nn.Conv1d(embedding_dim, 128, 5, padding=2)
        self.conv2=torch.nn.Conv1d(embedding_dim, 128, 3, padding=1)
        self.conv3=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.conv4=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.conv5=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.fc = nn.Linear(256, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.crf_flag=crf
        if self.crf_flag:
            self.crf = ConditionalRandomField(output_dim) 
        
    def forward(self, ids, ids_len, tag, is_training=True):
        # ids = [batch size, seq len]
#         print(f"ids: {ids.size()}")
        embedded = self.dropout(self.embedding(ids))
#         print(f"embedding: {embedded.size()}")
        # embedded = [batch size, seq len, embedding dim]
        conved = embedded.transpose(1, 2)
#         print(f"after permute: {conved.size()}")
        # embedded = [batch size, embedding dim, seq len]
        x_conv=torch.nn.functional.relu(torch.cat((self.conv1(conved), self.conv2(conved)), dim=1))
#         x_conv=torch.nn.functional.relu(self.conv1(conved))
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv3(x_conv))
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv4(x_conv))
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv5(x_conv))
        conved=x_conv.transpose(1, 2)
        logit = self.fc(conved)
        if not is_training:
            if self.crf_flag:
                score = self.crf.viterbi_tags(logit)
            else:
                x_logit = logit.transpose(2, 0)
                score = torch.nn.functional.log_softmax(x_logit).transpose(2, 0)
        else:
            if self.crf_flag:
                score = - self.crf(logit, tag)
            else:
                x_logit = torch.nn.utils.rnn.pack_padded_sequence(logit, ids_len, batch_first=True)
                score = torch.nn.functional.nll_loss(torch.nn.functional.log_softmax(x_logit.data), tag.data)
        return score

time: 2.29 ms (started: 2022-08-11 20:50:14 +00:00)


In [16]:
import torchtext
def tokenize_example(example, max_length=300):
    return example.split(' ')[:max_length]
def split_tags(example, max_length=300):
    return example.split(',')[:max_length]

# data2 = data2.sample(200)

data2['tokens'] = data2['sentence'].map(tokenize_example)
data2['labels'] = data2['word_labels2'].map(split_tags)

# K-fold
k_fold = 5
cur_fold = 3 # [3]
gap = int(len(data2) / k_fold)
indices = [[i*gap, (i+1)*gap if i+1 != k_fold else len(data2)] 
          for i in range(k_fold)]
data3 = data2.sample(frac=1, random_state=200)
test_dataset = data3[indices[cur_fold][0]:indices[cur_fold][1]]
train_dataset = data3.drop(test_dataset.index).reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)

min_freq = 3
special_tokens = ['<unk>', '<pad>']

vocab = torchtext.vocab.build_vocab_from_iterator(train_dataset['tokens'],
                                                  min_freq=min_freq,
                                                  specials=special_tokens)

time: 2.2 s (started: 2022-08-11 20:50:14 +00:00)


In [17]:
unk_index = vocab['<unk>']
pad_index = vocab['<pad>']
vocab.set_default_index(unk_index)

def numericalize_data(example, vocab=vocab):
    return [vocab[token] for token in example]

def numericalize_label(example, vocab=labels_to_ids2):
    return [vocab[token] for token in example]

train_dataset['ids'] = train_dataset['tokens'].map(numericalize_data)
train_dataset['labels2'] = train_dataset['labels'].map(numericalize_label)

time: 3.09 s (started: 2022-08-11 20:50:17 +00:00)


In [18]:
vocab_size = len(vocab)
embedding_dim = 300
# n_filters = 100
# filter_sizes = [5,3,5,5]
output_dim = len(labels_to_ids2)
dropout_rate = 0.5

model = CNN_n(vocab_size, embedding_dim, output_dim, dropout_rate, pad_index, crf=True)

time: 111 ms (started: 2022-08-11 20:50:20 +00:00)


In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,042,800 trainable parameters
time: 793 µs (started: 2022-08-11 20:50:20 +00:00)


In [20]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Conv1d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        nn.init.zeros_(m.bias)

time: 1.12 ms (started: 2022-08-11 20:50:20 +00:00)


In [21]:
model.apply(initialize_weights)

CNN_n(
  (embedding): Embedding(32498, 300, padding_idx=1)
  (conv1): Conv1d(300, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv2): Conv1d(300, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv4): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv5): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (fc): Linear(in_features=256, out_features=8, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (crf): ConditionalRandomField()
)

time: 16.5 ms (started: 2022-08-11 20:50:20 +00:00)


In [22]:
vectors = torchtext.vocab.FastText()

.vector_cache/wiki.en.vec: 6.60GB [03:50, 28.6MB/s]                                
100%|██████████| 2519370/2519370 [04:33<00:00, 9218.58it/s]


time: 9min 19s (started: 2022-08-11 20:50:20 +00:00)


In [23]:
pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())

time: 275 ms (started: 2022-08-11 20:59:39 +00:00)


In [24]:
model.embedding.weight.data = pretrained_embedding
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
# criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# criterion = criterion.to(device)

time: 107 ms (started: 2022-08-11 20:59:39 +00:00)


In [25]:
from torch.nn.utils.rnn import pad_sequence
def sent2ids(text):
    return [vocab[w] for w in text]

def labels2ids(labels):
    return [labels_to_ids2[w] for w in labels]

def data_generator(sents, labels, batch_size=32, is_training=True, index=0):
    if is_training:
        select_indices = np.random.choice(len(sents), batch_size, replace=False)
    else:
        start = index
        end = min(start + batch_size, len(sents)) 
        select_indices = list(range(start, end))
    #select_indices = list(range(batch_size))
    batch_sents = np.array(sents)[select_indices]
    batch_labels = np.array(labels)[select_indices]
    
    batch_sents = list(map(sent2ids, batch_sents))
    batch_labels = list(map(labels2ids, batch_labels))
    
    seq_lens = [len(s) for s in batch_sents]
    seq_lens = torch.LongTensor(seq_lens)
    max_len = max(seq_lens)
    
    batch_sents = [torch.LongTensor(s) for s in batch_sents]
    
    batch_sents = pad_sequence(batch_sents, batch_first=True)

    if not is_training:
        return batch_sents, batch_labels, seq_lens, end
    batch_labels = [torch.LongTensor(s) for s in batch_labels]
    batch_labels = pad_sequence(batch_labels, batch_first=True)
  
    return batch_sents, batch_labels, seq_lens

time: 1.64 ms (started: 2022-08-11 20:59:40 +00:00)


In [26]:
import time

train_sent_words = train_dataset['tokens']
train_sent_tags = train_dataset['labels']
epoch = 40
loop_num = int(len(train_sent_words)/100)
print(f"The loop numbers: {loop_num}")
for i in range(epoch):
    print(f"Epoch {i}:")
    start = time.time()
    model.train()
    for j in range(loop_num):
        optimizer.zero_grad()
        batch_sents, batch_tags, seq_lens, = data_generator(train_sent_words, train_sent_tags, batch_size=100)
        loss = model(batch_sents.to(device), seq_lens.to(device), batch_tags.to(device))
        loss.backward()
        optimizer.step()
        if j % 100 == 0:
            print(f'Loss: {loss.item()} \t Cost time per step: {time.time() - start}')

The loop numbers: 263
Epoch 0:
Loss: 61739.8125 	 Cost time per step: 7.18489146232605
Loss: 1653.6947021484375 	 Cost time per step: 35.791404485702515
Loss: 734.48193359375 	 Cost time per step: 64.74219727516174
Epoch 1:
Loss: 742.416748046875 	 Cost time per step: 0.279463529586792
Loss: 594.3251953125 	 Cost time per step: 28.63482689857483
Loss: 442.1259765625 	 Cost time per step: 57.19734048843384
Epoch 2:
Loss: 452.62451171875 	 Cost time per step: 0.2781364917755127
Loss: 365.93505859375 	 Cost time per step: 28.850520372390747
Loss: 306.5361328125 	 Cost time per step: 57.527618169784546
Epoch 3:
Loss: 361.11669921875 	 Cost time per step: 0.2790236473083496
Loss: 325.17822265625 	 Cost time per step: 28.39827275276184
Loss: 209.03857421875 	 Cost time per step: 56.48990345001221
Epoch 4:
Loss: 243.12548828125 	 Cost time per step: 0.2745969295501709
Loss: 267.0947265625 	 Cost time per step: 28.796079635620117
Loss: 200.029296875 	 Cost time per step: 57.55385661125183
Epoc

In [27]:
def tags_to_keywords(sample, words):
    indices = [i for i, l in enumerate(sample) if l != 'O']
    keywords, key_cats = [], []
    for j, id in enumerate(indices):
        if j == 0:
            start = end = id
            continue
        if j == len(indices):
            pos = pos_tag(words[start:end+1])
            if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
                keywords.append(' '.join(words[start:end+1]))
                key_cats.append((' '.join(words[start:end+1]), sample[start:end+1]))
            continue
        if end+1 == id:
            end = id
        else:
            try:
                pos = pos_tag(words[start:end+1])
                if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
                    keywords.append(' '.join(words[start:end+1]))
                    key_cats.append((' '.join(words[start:end+1]), sample[start:end+1]))
                start = end = id
            except:
                continue
    return list(set(keywords)), key_cats

time: 2.05 ms (started: 2022-08-11 21:49:34 +00:00)


In [28]:
dev_sent_words = test_dataset['tokens']
dev_sent_tags = test_dataset['labels']
model.eval()
index = 0
pred_label_list = []
Preds, Preds_cats = [], []
pbar = tqdm.tqdm(total=len(dev_sent_words))
while index < len(dev_sent_words):
    batch_sents, batch_tags, seq_lens, index = data_generator(dev_sent_words, 
                                                              dev_sent_tags, batch_size=32, 
                                                              is_training=False, index=index)
    pred_labels = model(batch_sents.to(device), seq_lens.to(device), batch_tags, is_training=False)
    for i, label_seq in enumerate(pred_labels):
        pred_labels = [ids_to_labels2[t] for t in label_seq[0]]
        
        preds, preds_cats = tags_to_keywords(pred_labels, list(dev_sent_words[index-32:min(index, len(dev_sent_words))])[i])
        
        pred_label_list.append(pred_labels)
        Preds.append(preds)
        Preds_cats.append(preds_cats)
    
    pbar.update(1000)
pbar.close()

206000it [02:09, 1596.72it/s]                        

time: 2min 9s (started: 2022-08-11 21:49:34 +00:00)


In [29]:
predictions = [p for pl in pred_label_list for p in pl]
labels = [d for dev in dev_sent_tags for d in dev]

time: 128 ms (started: 2022-08-11 21:51:43 +00:00)


In [30]:
pred_lable_keywords = {}
pred_lable_keywords['tag_preds'] = predictions
pred_lable_keywords['tag_labels'] = labels
pred_lable_keywords['predicitons'] = Preds
pred_lable_keywords['preds_cats'] = Preds_cats
pred_lable_keywords['cnn_pred_label_list'] = pred_label_list

time: 692 µs (started: 2022-08-11 21:51:43 +00:00)


In [31]:
print("Saving results and models ... ")
with open(f"./CNN-label-pred-keywords-kfold{cur_fold}.pkl", "wb") as f:
    pickle.dump(pred_lable_keywords, f)

torch.save(model.state_dict, f"./Model_state_dict-kfold{cur_fold}.pt")

torch.save(model, f"./CNN-kfold{cur_fold}.pt")

Saving results and models ... 
time: 369 ms (started: 2022-08-11 21:51:43 +00:00)


### CNN

In [9]:
import functools
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda
time: 67.2 ms (started: 2022-08-03 14:14:23 +00:00)


In [10]:
seed = 0

torch.manual_seed(seed)

time: 5.23 ms (started: 2022-08-03 14:14:24 +00:00)


In [11]:
class CNN_n(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, dropout_rate, pad_index, crf=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        self.conv1=torch.nn.Conv1d(embedding_dim, 128, 5, padding=2)
        self.conv2=torch.nn.Conv1d(embedding_dim, 128, 3, padding=1)
        self.conv3=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.conv4=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.conv5=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.fc = nn.Linear(256, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.crf_flag=crf
        if self.crf_flag:
            self.crf = ConditionalRandomField(output_dim) 
        
    def forward(self, ids, ids_len, mask, tag=None, is_training=True):
        # ids = [batch size, seq len]
#         print(f"ids: {ids.size()}")
        embedded = self.dropout(self.embedding(ids))
#         print(f"embedding: {embedded.size()}")
        # embedded = [batch size, seq len, embedding dim]
        conved = embedded.transpose(1, 2)
#         print(f"after permute: {conved.size()}")
        # embedded = [batch size, embedding dim, seq len]
        x_conv=torch.nn.functional.relu(torch.cat((self.conv1(conved), self.conv2(conved)), dim=1))
#         x_conv=torch.nn.functional.relu(self.conv1(conved))
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv3(x_conv))
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv4(x_conv))
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv5(x_conv))
        conved=x_conv.transpose(1, 2)
        logit = self.fc(conved)
        if not is_training:
            if self.crf_flag:
                score = self.crf.viterbi_tags(logit, mask)
            else:
                x_logit = logit.transpose(2, 0)
                score = torch.nn.functional.log_softmax(x_logit).transpose(2, 0)
        else:
            if self.crf_flag:
                score = - self.crf(logit, tag, mask)
            else:
                x_logit = torch.nn.utils.rnn.pack_padded_sequence(logit, ids_len, batch_first=True)
                score = torch.nn.functional.nll_loss(torch.nn.functional.log_softmax(x_logit.data), tag.data)
        return score

time: 2.89 ms (started: 2022-08-03 14:14:26 +00:00)


In [12]:
import torchtext
def tokenize_example(example, max_length=300):
    return example.split(' ')[:max_length]
def split_tags(example, max_length=300):
    return example.split(',')[:max_length]

# data2 = data2.sample(200)

data2['tokens'] = data2['sentence'].map(tokenize_example)
data2['labels'] = data2['word_labels2'].map(split_tags)

train_size = 0.8
train_dataset = data2.sample(frac=train_size,random_state=200)
test_dataset = data2.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

min_freq = 3
special_tokens = ['<unk>', '<pad>']

vocab = torchtext.vocab.build_vocab_from_iterator(train_dataset['tokens'],
                                                  min_freq=min_freq,
                                                  specials=special_tokens)

time: 2.39 s (started: 2022-08-03 14:14:28 +00:00)


In [13]:
unk_index = vocab['<unk>']
pad_index = vocab['<pad>']
vocab.set_default_index(unk_index)

def numericalize_data(example, vocab=vocab):
    return [vocab[token] for token in example]

def numericalize_label(example, vocab=labels_to_ids2):
    return [vocab[token] for token in example]

train_dataset['ids'] = train_dataset['tokens'].map(numericalize_data)
train_dataset['labels2'] = train_dataset['labels'].map(numericalize_label)

time: 3.1 s (started: 2022-08-03 14:14:31 +00:00)


In [14]:
from allennlp.modules import ConditionalRandomField

time: 543 ms (started: 2022-08-03 14:14:34 +00:00)


In [15]:
vocab_size = len(vocab)
embedding_dim = 300
output_dim = len(labels_to_ids2)
dropout_rate = 0.5

model = CNN_n(vocab_size, embedding_dim, output_dim, dropout_rate, pad_index, crf=True)

time: 113 ms (started: 2022-08-03 14:14:34 +00:00)


In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,112,700 trainable parameters
time: 833 µs (started: 2022-08-03 14:14:34 +00:00)


In [17]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Conv1d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        nn.init.zeros_(m.bias)

time: 3.39 ms (started: 2022-08-03 14:14:37 +00:00)


In [18]:
model.apply(initialize_weights)

CNN_n(
  (embedding): Embedding(32731, 300, padding_idx=1)
  (conv1): Conv1d(300, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv2): Conv1d(300, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv4): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv5): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (fc): Linear(in_features=256, out_features=8, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (crf): ConditionalRandomField()
)

time: 15.1 ms (started: 2022-08-03 14:14:38 +00:00)


In [21]:
vectors = torchtext.vocab.GloVe()

.vector_cache/glove.840B.300d.zip: 2.18GB [06:51, 5.30MB/s]                               
100%|█████████▉| 2196016/2196017 [04:03<00:00, 9002.64it/s]


time: 12min 49s (started: 2022-08-03 14:18:48 +00:00)


In [22]:
pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())
model.embedding.weight.data = pretrained_embedding
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
model = model.to(device)

time: 460 ms (started: 2022-08-03 14:32:39 +00:00)


In [26]:
from torch.nn.utils.rnn import pad_sequence
def sent2ids(text):
    return [vocab[w] for w in text]

def labels2ids(labels):
    return [labels_to_ids2[w] for w in labels]

def data_generator(sents, labels, batch_size=32, is_training=True, index=0):
    if is_training:
        select_indices = np.random.choice(len(sents), batch_size, replace=False)
    else:
        start = index
        end = min(start + batch_size, len(sents)) 
        select_indices = list(range(start, end))
    #select_indices = list(range(batch_size))
    batch_sents = np.array(sents)[select_indices]
    batch_labels = np.array(labels)[select_indices]
    
    batch_sents = list(map(sent2ids, batch_sents))
    batch_labels = list(map(labels2ids, batch_labels))
    
    seq_lens = [len(s) for s in batch_sents]
    seq_lens = torch.LongTensor(seq_lens)
    max_len = max(seq_lens)
    
    batch_sents = [torch.LongTensor(s) for s in batch_sents]
    
    batch_sents = pad_sequence(batch_sents, batch_first=True)
    
    batch_labels = [torch.LongTensor(s) for s in batch_labels]
    batch_labels = pad_sequence(batch_labels, batch_first=True)

    batch_mask = (batch_sents!=0).numpy().astype(np.uint8)
    batch_mask = torch.from_numpy(batch_mask)
    
    if not is_training:
        return batch_sents, batch_labels, seq_lens, batch_mask, end
    return batch_sents, batch_labels, seq_lens, batch_mask

time: 4.46 ms (started: 2022-08-03 14:33:32 +00:00)


In [29]:
import time

train_sent_words = train_dataset['tokens']
train_sent_tags = train_dataset['labels']
epoch = 10
loop_num = int(len(train_sent_words)/100)
print(f"The loop numbers: {loop_num}")
for i in range(epoch):
    start = time.time()
    model.train()
    for j in range(loop_num):
        optimizer.zero_grad()
        batch_sents, batch_tags, seq_lens, batch_mask = data_generator(train_sent_words, train_sent_tags, batch_size=100)
        loss = model(batch_sents.to(device), seq_lens.to(device), batch_mask.to(device), batch_tags.to(device))
        loss.backward()
        optimizer.step()
        if j % 100 == 0:
            print(f'Loss: {loss.item()} \t Cost time per step: {time.time() - start}')

The loop numbers: 263


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

time: 56.7 ms (started: 2022-08-03 14:34:01 +00:00)


In [28]:
CUDA_LAUNCH_BLOCKING=1

time: 402 µs (started: 2022-08-03 14:33:57 +00:00)


In [39]:
dev_sent_words = test_dataset['tokens']
dev_sent_tags = test_dataset['labels']
model.eval()
index = 0
pred_label_list = []
while index < len(dev_sent_words):
    batch_sents, batch_tags, seq_lens, batch_mask, index = data_generator(dev_sent_words, 
                                                              dev_sent_tags, batch_size=32, 
                                                              is_training=False, index=index)
    pred_labels = model(batch_sents.to(device), seq_lens.to(device), batch_mask.to(device), batch_tags, is_training=False)
    for label_seq in pred_labels:
        pred_labels = [ids_to_labels2[t] for t in label_seq[0]]
        pred_label_list.append(pred_labels)

KeyError: 9

time: 380 ms (started: 2022-08-03 13:58:55 +00:00)


### Other Model - 1

In [25]:
import time
import torch
import torch.optim as optim
import os
import sys
from torch.nn.utils.rnn import pack_padded_sequence
from sklearn.metrics import f1_score

time: 961 µs (started: 2022-09-09 09:13:51 +00:00)


In [26]:
# emb_file = '../input/glove6b100dtxt/glove.6B.100d.txt'  # path to pre-trained word embeddings
emb_file = '../input/glove6b100dtxt/glove.6B.100d.txt'
min_word_freq = 3  # threshold for word frequency
min_char_freq = 1  # threshold for character frequency
caseless = True  # lowercase everything?
expand_vocab = True  # expand model's input vocabulary to the pre-trained embeddings' vocabulary?

# Model parameters
char_emb_dim = 30  # character embedding size
with open(emb_file, 'r') as f:
    word_emb_dim = len(f.readline().split(' ')) - 1  # word embedding size
word_rnn_dim = 300  # word RNN size
char_rnn_dim = 300  # character RNN size
char_rnn_layers = 1  # number of layers in character RNN
word_rnn_layers = 1  # number of layers in word RNN
highway_layers = 1  # number of layers in highway network
dropout = 0.5  # dropout
fine_tune_word_embeddings = False  # fine-tune pre-trained word embeddings?

# Training parameters
start_epoch = 0  # start at this epoch
batch_size = 32  # batch size
lr = 0.015  # learning rate
lr_decay = 0.05  # decay learning rate by this amount
momentum = 0.9  # momentum
workers = 1  # number of workers for loading data in the DataLoader
epochs = 12  # number of epochs to run without early-stopping
grad_clip = 5.  # clip gradients at this value
print_freq = 100  # print training or validation status every __ batches
best_f1 = 0.  # F1 score to start with
checkpoint = None  # path to model checkpoint, None if none

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time: 12.4 ms (started: 2022-09-09 09:13:57 +00:00)


In [27]:
def tokenize_example(example, max_length=300):
    return example.split(' ')[:max_length]

def split_tags(example, max_length=300):
    return example.split(',')[:max_length]

# data2 = data2.sample(200)

data2['tokens'] = data2['sentence'].map(tokenize_example)
data2['labels'] = data2['word_labels2'].map(split_tags)

time: 1.21 s (started: 2022-09-09 09:13:59 +00:00)


In [28]:
# train_size = 0.8
# train_dataset2 = data2.sample(frac=train_size, random_state=200)
# test_dataset2 = data2.drop(train_dataset2.index).reset_index(drop=True)
# train_dataset2 = train_dataset2.reset_index(drop=True)
k_fold = 5
cur_fold = 3
gap = int(len(data2) / k_fold)
indices = [[i*gap, (i+1)*gap if i+1 != k_fold else len(data2)] 
          for i in range(k_fold)]
data3 = data2.sample(frac=1, random_state=200)
test_dataset2 = data3[indices[cur_fold][0]:indices[cur_fold][1]]
train_dataset2 = data3.drop(test_dataset2.index).reset_index(drop=True)
test_dataset2 = test_dataset2.reset_index(drop=True)

time: 550 ms (started: 2022-09-09 09:14:04 +00:00)


In [29]:
word_map, char_map, tag_map = create_maps(data2.tokens, data2.labels, min_word_freq,
                                                  min_char_freq)

time: 1min 19s (started: 2022-09-09 09:14:05 +00:00)


In [30]:
embeddings, word_map, lm_vocab_size = load_embeddings(emb_file, word_map,
                                                              expand_vocab)

Embedding length is 100.
You have elected to include embeddings that are out-of-corpus.

Loading embeddings...
'word_map' is being updated accordingly.

Done.
 Embedding vocabulary: 415143
 Language Model vocabulary: 30184.

time: 28.6 s (started: 2022-09-09 09:15:25 +00:00)


In [31]:
model = LM_LSTM_CRF(tagset_size=len(tag_map),
                    charset_size=len(char_map),
                    char_emb_dim=char_emb_dim,
                    char_rnn_dim=char_rnn_dim,
                    char_rnn_layers=char_rnn_layers,
                    vocab_size=len(word_map),
                    lm_vocab_size=lm_vocab_size,
                    word_emb_dim=word_emb_dim,
                    word_rnn_dim=word_rnn_dim,
                    word_rnn_layers=word_rnn_layers,
                    dropout=dropout,
                    highway_layers=highway_layers).to(device)
model.init_word_embeddings(embeddings.to(device))  # initialize embedding layer with pre-trained embeddings
model.fine_tune_word_embeddings(fine_tune_word_embeddings)  # fine-tune
optimizer = optim.SGD(params=filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)


time: 580 ms (started: 2022-09-09 09:15:53 +00:00)


In [32]:
# Loss functions
lm_criterion = nn.CrossEntropyLoss().to(device)
crf_criterion = ViterbiLoss(tag_map).to(device)

# Since the language model's vocab is restricted to in-corpus indices, encode training/val with only these!
# word_map might have been expanded, and in-corpus words eliminated due to low frequency might still be added because
# they were in the pre-trained embeddings
temp_word_map = {k: v for k, v in word_map.items() if v <= word_map['<unk>']}
train_inputs = create_input_tensors(train_dataset2.tokens, train_dataset2.labels, temp_word_map, char_map,
                                    tag_map)
val_inputs = create_input_tensors(test_dataset2.tokens, test_dataset2.labels, temp_word_map, char_map, tag_map)

# DataLoaders
train_loader = torch.utils.data.DataLoader(WCDataset(*train_inputs), batch_size=batch_size, shuffle=True,
                                           num_workers=workers, pin_memory=False)
val_loader = torch.utils.data.DataLoader(WCDataset(*val_inputs), batch_size=batch_size, shuffle=True,
                                         num_workers=workers, pin_memory=False)

# Viterbi decoder (to find accuracy during validation)
vb_decoder = ViterbiDecoder(tag_map)

time: 3min 16s (started: 2022-09-09 09:15:54 +00:00)


In [33]:

def train(train_loader, model, lm_criterion, crf_criterion, optimizer, epoch, vb_decoder):
    """
    Performs one epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param lm_criterion: cross entropy loss layer
    :param crf_criterion: viterbi loss layer
    :param optimizer: optimizer
    :param epoch: epoch number
    :param vb_decoder: viterbi decoder (to decode and find F1 score)
    """

    model.train()  # training mode enables dropout

    ce_losses = AverageMeter()  # cross entropy loss
    vb_losses = AverageMeter()  # viterbi loss
    f1s = AverageMeter()  # f1 score

    start = time.time()

    # Batches
    for i, (wmaps, cmaps_f, cmaps_b, cmarkers_f, cmarkers_b, tmaps, wmap_lengths, cmap_lengths) in enumerate(
            train_loader):

        max_word_len = max(wmap_lengths.tolist())
        max_char_len = max(cmap_lengths.tolist())

        # Reduce batch's padded length to maximum in-batch sequence
        # This saves some compute on nn.Linear layers (RNNs are unaffected, since they don't compute over the pads)
        wmaps = wmaps[:, :max_word_len].to(device)
        cmaps_f = cmaps_f[:, :max_char_len].to(device)
        cmaps_b = cmaps_b[:, :max_char_len].to(device)
        cmarkers_f = cmarkers_f[:, :max_word_len].to(device)
        cmarkers_b = cmarkers_b[:, :max_word_len].to(device)
        tmaps = tmaps[:, :max_word_len].to(device)
        wmap_lengths = wmap_lengths.to(device)
        cmap_lengths = cmap_lengths.to(device)

        # Forward prop.
        crf_scores, lm_f_scores, lm_b_scores, wmaps_sorted, tmaps_sorted, wmap_lengths_sorted, _, __ = model(cmaps_f,
                                                                                                             cmaps_b,
                                                                                                             cmarkers_f,
                                                                                                             cmarkers_b,
                                                                                                             wmaps,
                                                                                                             tmaps,
                                                                                                             wmap_lengths,
                                                                                                             cmap_lengths)

        # LM loss

        # We don't predict the next word at the pads or <end> tokens
        # We will only predict at [dunston, checks, in] among [dunston, checks, in, <end>, <pad>, <pad>, ...]
        # So, prediction lengths are word sequence lengths - 1
        lm_lengths = wmap_lengths_sorted - 1
        lm_lengths = lm_lengths.tolist()

        # Remove scores at timesteps we won't predict at
        # pack_padded_sequence is a good trick to do this (see dynamic_rnn.py, where we explore this)
        lm_f_scores = pack_padded_sequence(lm_f_scores, lm_lengths, batch_first=True)
        lm_b_scores = pack_padded_sequence(lm_b_scores, lm_lengths, batch_first=True)

        # For the forward sequence, targets are from the second word onwards, up to <end>
        # (timestep -> target) ...dunston -> checks, ...checks -> in, ...in -> <end>
        lm_f_targets = wmaps_sorted[:, 1:]
        lm_f_targets = pack_padded_sequence(lm_f_targets, lm_lengths, batch_first=True)

        # For the backward sequence, targets are <end> followed by all words except the last word
        # ...notsnud -> <end>, ...skcehc -> dunston, ...ni -> checks
        lm_b_targets = torch.cat(
            [torch.LongTensor([word_map['<end>']] * wmaps_sorted.size(0)).unsqueeze(1).to(device), wmaps_sorted], dim=1)
        lm_b_targets = pack_padded_sequence(lm_b_targets, lm_lengths, batch_first=True)

        # Calculate loss
        ce_loss = lm_criterion(lm_f_scores.data, lm_f_targets.data) + lm_criterion(lm_b_scores.data, lm_b_targets.data)
        wmap_lengths_sorted = wmap_lengths_sorted.to('cpu')
        vb_loss = crf_criterion(crf_scores, tmaps_sorted, wmap_lengths_sorted)
        loss = ce_loss + vb_loss

        # Back prop.
        optimizer.zero_grad()
        loss.backward()

        if grad_clip is not None:
            clip_gradient(optimizer, grad_clip)

        optimizer.step()

        # Viterbi decode to find accuracy / f1
        decoded = vb_decoder.decode(crf_scores.to("cpu"), wmap_lengths_sorted.to("cpu"))

        # Remove timesteps we won't predict at, and also <end> tags, because to predict them would be cheating
        decoded = pack_padded_sequence(decoded, lm_lengths, batch_first=True)
        tmaps_sorted = tmaps_sorted % vb_decoder.tagset_size  # actual target indices (see create_input_tensors())
        tmaps_sorted = pack_padded_sequence(tmaps_sorted, lm_lengths, batch_first=True)

        # F1
        f1 = f1_score(tmaps_sorted.data.to("cpu"), decoded.data.numpy(), average='macro')

        # Keep track of metrics
        ce_losses.update(ce_loss.item(), sum(lm_lengths))
        vb_losses.update(vb_loss.item(), crf_scores.size(0))
        f1s.update(f1, sum(lm_lengths))

    # Print training status
    print(f'Epoch: [{epoch}]\t Epoch Time {time.time() - start} \t'
          f'CE Loss {ce_losses.val:.4f} ({ce_losses.avg:.4f})\t'
          f'VB Loss {vb_losses.val:.4f} ({vb_losses.avg:.4f})\t'
          f'F1 {f1s.val:.3f} ({f1s.avg:.3f})')


def validate(val_loader, model, crf_criterion, vb_decoder):
    """
    Performs one epoch's validation.

    :param val_loader: DataLoader for validation data
    :param model: model
    :param crf_criterion: viterbi loss layer
    :param vb_decoder: viterbi decoder
    :return: validation F1 score
    """
    model.eval()

    batch_time = AverageMeter()
    vb_losses = AverageMeter()
    f1s = AverageMeter()
    labels, predictions = [], []

    start = time.time()

    for i, (wmaps, cmaps_f, cmaps_b, cmarkers_f, cmarkers_b, tmaps, wmap_lengths, cmap_lengths) in enumerate(
            val_loader):

        max_word_len = max(wmap_lengths.tolist())
        max_char_len = max(cmap_lengths.tolist())

        # Reduce batch's padded length to maximum in-batch sequence
        # This saves some compute on nn.Linear layers (RNNs are unaffected, since they don't compute over the pads)
        wmaps = wmaps[:, :max_word_len].to(device)
        cmaps_f = cmaps_f[:, :max_char_len].to(device)
        cmaps_b = cmaps_b[:, :max_char_len].to(device)
        cmarkers_f = cmarkers_f[:, :max_word_len].to(device)
        cmarkers_b = cmarkers_b[:, :max_word_len].to(device)
        tmaps = tmaps[:, :max_word_len].to(device)
        wmap_lengths = wmap_lengths.to(device)
        cmap_lengths = cmap_lengths.to(device)

        # Forward prop.
        crf_scores, wmaps_sorted, tmaps_sorted, wmap_lengths_sorted, _, __ = model(cmaps_f,
                                                                                   cmaps_b,
                                                                                   cmarkers_f,
                                                                                   cmarkers_b,
                                                                                   wmaps,
                                                                                   tmaps,
                                                                                   wmap_lengths,
                                                                                   cmap_lengths)

        # Viterbi / CRF layer loss
        vb_loss = crf_criterion(crf_scores, tmaps_sorted, wmap_lengths_sorted.to("cpu"))

        # Viterbi decode to find accuracy / f1
        decoded = vb_decoder.decode(crf_scores.to("cpu"), wmap_lengths_sorted.to("cpu"))

        # Remove timesteps we won't predict at, and also <end> tags, because to predict them would be cheating
        decoded = pack_padded_sequence(decoded, (wmap_lengths_sorted - 1).tolist(), batch_first=True)
        tmaps_sorted = tmaps_sorted % vb_decoder.tagset_size  # actual target indices (see create_input_tensors())
        tmaps_sorted = pack_padded_sequence(tmaps_sorted, (wmap_lengths_sorted - 1).tolist(), batch_first=True)

        # f1
        f1 = f1_score(tmaps_sorted.data.to("cpu"), decoded.data.numpy(), average='macro')
        
        labels.append(tmaps_sorted.data)
        predictions.append(decoded.data)
        # Keep track of metrics
        vb_losses.update(vb_loss.item(), crf_scores.size(0))
        f1s.update(f1, sum((wmap_lengths_sorted - 1).tolist()))
        batch_time.update(time.time() - start)

        start = time.time()

        if i % print_freq == 0:
            print('Validation: [{0}/{1}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'VB Loss {vb_loss.val:.4f} ({vb_loss.avg:.4f})\t'
                  'F1 Score {f1.val:.3f} ({f1.avg:.3f})\t'.format(i, len(val_loader), batch_time=batch_time,
                                                                  vb_loss=vb_losses, f1=f1s))

    print(
        '\n * LOSS - {vb_loss.avg:.3f}, F1 SCORE - {f1.avg:.3f}\n'.format(vb_loss=vb_losses,
                                                                          f1=f1s))

    return f1s.avg, labels, predictions

time: 8.59 ms (started: 2022-09-09 09:19:10 +00:00)


In [34]:
for epoch in range(start_epoch, epochs):

    # One epoch's training
    train(train_loader=train_loader,
          model=model,
          lm_criterion=lm_criterion,
          crf_criterion=crf_criterion,
          optimizer=optimizer,
          epoch=epoch,
          vb_decoder=vb_decoder)

    # One epoch's validation
    val_f1, labels, predictions = validate(val_loader=val_loader,
                      model=model,
                      crf_criterion=crf_criterion,
                      vb_decoder=vb_decoder)

    # Did validation F1 score improve?
    is_best = val_f1 > best_f1
    best_f1 = max(val_f1, best_f1)
    if not is_best:
        epochs_since_improvement += 1
        print("\nEpochs since improvement: %d\n" % (epochs_since_improvement,))
    else:
        epochs_since_improvement = 0
        
    # Decay learning rate every epoch
    adjust_learning_rate(optimizer, lr / (1 + (epoch + 1) * lr_decay))

Epoch: [0]	 Epoch Time 1023.2107527256012 	CE Loss 14.7385 (15.9708)	VB Loss 161.5673 (203.0695)	F1 0.165 (0.161)
Validation: [0/206]	Batch Time 0.849 (0.849)	VB Loss 181.2461 (181.2461)	F1 Score 0.165 (0.165)	
Validation: [100/206]	Batch Time 0.468 (0.491)	VB Loss 190.3750 (171.5852)	F1 Score 0.165 (0.176)	
Validation: [200/206]	Batch Time 0.467 (0.494)	VB Loss 218.4609 (173.1048)	F1 Score 0.165 (0.176)	

 * LOSS - 173.513, F1 SCORE - 0.176


DECAYING learning rate.
The new learning rate is 0.014286

Epoch: [1]	 Epoch Time 1023.7744996547699 	CE Loss 14.5567 (14.6788)	VB Loss 150.9423 (147.5731)	F1 0.165 (0.154)
Validation: [0/206]	Batch Time 0.885 (0.885)	VB Loss 100.4375 (100.4375)	F1 Score 0.165 (0.165)	
Validation: [100/206]	Batch Time 0.467 (0.513)	VB Loss 183.1562 (122.5514)	F1 Score 0.197 (0.177)	
Validation: [200/206]	Batch Time 0.463 (0.505)	VB Loss 110.2188 (121.2852)	F1 Score 0.198 (0.177)	

 * LOSS - 121.174, F1 SCORE - 0.176


DECAYING learning rate.
The new learning rate

KeyboardInterrupt: 

time: 45min 10s (started: 2022-09-09 09:19:10 +00:00)


In [23]:
len(labels), len(predictions)

(206, 206)

time: 3.46 ms (started: 2022-08-06 20:01:57 +00:00)


In [24]:
labels1 = [l.to("cpu").item() for ls in labels for l in ls]
predictions1 = [l.to("cpu").item() for ls in predictions for l in ls]

time: 20.3 s (started: 2022-08-06 20:02:06 +00:00)


In [25]:
tag_to_cat = {v:k for k, v in tag_map.items()}
labels2 = [tag_to_cat[l] for l in labels1]
predictions2 = [tag_to_cat[l] for l in predictions1]

time: 153 ms (started: 2022-08-06 20:02:26 +00:00)


In [99]:
words = []
for i, (wmaps, cmaps_f, cmaps_b, cmarkers_f, cmarkers_b, tmaps, wmap_lengths, cmap_lengths) in enumerate(
        val_loader):

    max_word_len = max(wmap_lengths.tolist())
    max_char_len = max(cmap_lengths.tolist())

    # Reduce batch's padded length to maximum in-batch sequence
    # This saves some compute on nn.Linear layers (RNNs are unaffected, since they don't compute over the pads)
    wmaps = wmaps[:, :max_word_len]
    words.append(wmaps)

time: 225 ms (started: 2022-08-01 16:15:03 +00:00)


In [26]:
dev_sent_words = test_dataset2['tokens']
words2 = [d for dev in dev_sent_words for d in dev]

time: 83.8 ms (started: 2022-08-06 20:02:26 +00:00)


In [27]:
len(words2), len(labels2)

(1101738, 1101738)

time: 3.78 ms (started: 2022-08-06 20:02:26 +00:00)


In [28]:
tag_to_word = {v:k for k, v in temp_word_map.items()}
words2 = [tag_to_word[l] for l in words1]

NameError: name 'words1' is not defined

time: 462 ms (started: 2022-08-06 20:02:30 +00:00)


In [29]:
def countKeywords(words, predictions, labels):
    kws_pairs = []
    for i in zip(words, predictions, labels):
        kws_pairs.append(i)
        
    predict_kws = []
    tmp_wd = []
    tmp_pred, tmp_gold = '', ''
    for word, pred, gold in kws_pairs:
        if pred != 'O':
            if len(tmp_wd) == 0: 
                tmp_pred = pred
            else:
                if tmp_pred != pred:
                    predict_kws.append((' '.join(tmp_wd), tmp_pred))
                    tmp_wd = []
                    tmp_pred = pred
            tmp_wd.append(word)
        else:
            if len(tmp_wd) > 0:
                predict_kws.append((' '.join(tmp_wd), tmp_pred))
                tmp_wd = []
                tmp_pred = pred

    # check whether there are new keywords
    gold_dict = build_gold_dict()
    new_kws, kw_tags = [], []
    gold_lists = list(gold_dict.keys())
    gold_lists = [g.lower() for g in gold_lists]
    for word, label in predict_kws:
        if not word.lower() in gold_lists:
            new_kws.append(word.lower())
            kw_tags.append(tag2cat[label])
    new_keywords = list(zip(new_kws, kw_tags))

    new_kws_dict, new_kws_pos = {}, []
    for kw, cat in new_keywords:
        pos = pos_tag(kw.split())
        if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
            key = ' '.join([p for p, t in pos])
            tag = [t for p, t in pos]
            new_kws_pos.append(key)
            if not key in new_kws_dict.keys():
                new_kws_dict[key] = (tag, cat)

    new_kws = list(new_kws_dict.keys())
    cats = [value[1] for value in new_kws_dict.values()]

    df = pd.DataFrame({"keyword": new_kws, "category": cats})

    s = df['keyword'].apply(lengthOfTokens)
    s.sort_values()

    predict_kws = Counter(predict_kws)
    new_kws_pos = Counter(new_kws_pos)

    predict_kws = sorted(predict_kws.items(), key=lambda pair: pair[1], reverse=True)
    new_kws_pos = sorted(new_kws_pos.items(), key=lambda pair: pair[1], reverse=True)

    return predict_kws, new_kws_pos, df

def lengthOfTokens(pair):
    return len(pair.split())

time: 2.95 ms (started: 2022-08-06 20:02:39 +00:00)


In [30]:
predict_kws, new_kws_pos, df = countKeywords(words2, predictions2, labels2)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  


time: 3.81 s (started: 2022-08-06 20:02:42 +00:00)


In [1]:
predict_kws

NameError: name 'predict_kws' is not defined

In [32]:
new_kws_pos

[('development', 126),
 ('business', 112),
 ('products', 81),
 ('product', 71),
 ('research', 51),
 ('sales', 50),
 ('management', 47),
 ('market', 46),
 ('company', 45),
 ('services', 44),
 ('data', 44),
 ('information', 43),
 ('future', 42),
 ('technology', 42),
 ('customers', 39),
 ('costs', 37),
 ('customer', 37),
 ('risks', 37),
 ('results', 35),
 ('year', 35),
 ('risk', 35),
 ('operations', 35),
 ('activities', 34),
 ('assets', 33),
 ('performance', 33),
 ('group', 33),
 ('value', 32),
 ('revenue', 31),
 ('solutions', 29),
 ('use', 26),
 ('expenses', 26),
 ('regulations', 26),
 ('property', 25),
 ('time', 25),
 ('software', 25),
 ('tax', 24),
 ('support', 24),
 ('continue', 24),
 ('energy', 24),
 ('companies', 24),
 ('impact', 24),
 ('operating', 24),
 ('report', 24),
 ('board', 24),
 ('marketing', 23),
 ('net', 23),
 ('ability', 23),
 ('statements', 22),
 ('change', 22),
 ('systems', 22),
 ('rights', 21),
 ('provide', 21),
 ('december', 21),
 ('requirements', 21),
 ('service', 2

time: 52.5 ms (started: 2022-08-02 11:10:16 +00:00)


In [32]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

def get_distribution(inputs, name):
    true_res = {}
    for k, v in tag2cat.items():
        indexlist = get_index_positions(inputs, k)
        left, right = 0, 0
        new_labels = []
        for i in indexlist:
            if left == 0:
                left = i
                right = i
                continue
            if right+1 == i:
                right = i
            else:
                new_labels.append((left, right))
                left = i
                right = i
        true_res[v] = new_labels
    print(f"The distribution of {name} is:")
    for k,v in true_res.items():
        print(f"There are {len(v)} occurances in {k}")
    return true_res

def get_confusion_matrix(true_labels, true_predictions):
    # strict mode
    tp_dict = {}
    for true_k, true_v in true_labels.items():
        true_total = len(true_v)
        tmp_dict = {}
        tmp_total = 0
        for pred_k, pred_v in true_predictions.items():
            num = 0
            for v in pred_v:
                if v in true_v:
                    num += 1
            tmp_dict[pred_k] = num
            tmp_total += num
        tmp_dict["O"] = true_total - tmp_total
        tp_dict[true_k] = tmp_dict
    tp_dict
    tmp_dict = {}
    for pred_k, pred_v in true_predictions.items():
        true_total = len(pred_v)
        num = 0
        for tp_k, tp_v in tp_dict.items():
            num += tp_v[pred_k]
        tmp_dict[pred_k] = true_total - num
    tmp_dict["O"] = 0
    tp_dict["O"] = tmp_dict
    return pd.DataFrame(tp_dict).T

time: 4.36 ms (started: 2022-08-06 20:04:08 +00:00)


In [33]:
true_labels = get_distribution(labels2, "labels")
true_predictions = get_distribution(predictions2, "predictions")
df = get_confusion_matrix(true_labels, true_predictions)
df

The distribution of labels is:
There are 3292 occurances in Sustainability preoccupations
There are 2312 occurances in Digital transformation
There are 409 occurances in Change in management
There are 11764 occurances in Innovation activities
There are 2659 occurances in Business Model
There are 41 occurances in Corporate social responsibility ou CSR
There are 3 occurances in marco-label
The distribution of predictions is:
There are 3818 occurances in Sustainability preoccupations
There are 2432 occurances in Digital transformation
There are 332 occurances in Change in management
There are 13661 occurances in Innovation activities
There are 2867 occurances in Business Model
There are 0 occurances in Corporate social responsibility ou CSR
There are 0 occurances in marco-label


,Sustainability preoccupations,Digital transformation,Change in management,Innovation activities,Business Model,Corporate social responsibility ou CSR,marco-label,O
Sustainability preoccupations,2681,0,0,1,0,0,0,610
Digital transformation,0,1740,0,0,0,0,0,572
Change in management,0,0,241,15,0,0,0,153
Innovation activities,2,0,6,10055,16,0,0,1685
Business Model,0,0,0,6,2565,0,0,88
Corporate social responsibility ou CSR,0,0,0,0,0,0,0,41
marco-label,0,0,0,0,3,0,0,0
O,1135,692,85,3584,283,0,0,0


time: 8.82 s (started: 2022-08-06 20:04:09 +00:00)


In [34]:
new_kws_dict = {}
new_kw_df = pd.DataFrame({})
new_kws_dict['predicted_keywords_lstm'] = predict_kws
new_kws_dict['new_keywords_epoch_lstm'] = new_kws_pos

new_kw_df = pd.concat([new_kw_df, df])
new_kw_df = new_kw_df.reset_index(drop=True)
print("The newly generated keywords and the predicted category corresponding to:")
print(new_kw_df)

The newly generated keywords and the predicted category corresponding to:
   Sustainability preoccupations  Digital transformation  \
0                           2681                       0   
1                              0                    1740   
2                              0                       0   
3                              2                       0   
4                              0                       0   
5                              0                       0   
6                              0                       0   
7                           1135                     692   

   Change in management  Innovation activities  Business Model  \
0                     0                      1               0   
1                     0                      0               0   
2                   241                     15               0   
3                     6                  10055              16   
4                     0                      6         

In [49]:
label_pred_dict = {}
label_pred_dict["prediction"] = predictions
label_pred_dict["labels"] = labels

with open("./label_pred_dict_1.pkl", "wb") as f:
    pickle.dump(label_pred_dict, f)

with open("./new_kws_dict_1.pkl", "wb") as f:
    pickle.dump(new_kws_dict, f)

new_kw_df.to_csv("./new-keywords-cate_1.csv")  

df.to_csv("./confusion-matrix_1.csv")

print("Saving models ... ")
torch.save(model.state_dict, "./Model_state_dict_1.pickle")

torch.save(model, "./model_1.pickle")

Saving models ... 
time: 1.53 s (started: 2022-08-06 20:33:35 +00:00)


In [46]:
!git clone https://github.com/Fan5Shi/KeywordExtraction.git

Cloning into 'KeywordExtraction'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 596 bytes | 596.00 KiB/s, done.
time: 1.92 s (started: 2022-08-06 20:31:56 +00:00)


In [51]:
!git init

Initialized empty Git repository in /kaggle/working/.git/
time: 1.16 s (started: 2022-08-06 20:33:55 +00:00)


In [53]:
!git add .

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> KeywordExtraction
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached KeywordExtraction
hint: 
hint: See "git help submodule" for more information.
time: 31.9 s (started: 2022-08-06 20:34:48 +00:00)


In [55]:
!git config --global user.email "963326056he@gmail.com"
!git config --global user.name "Fan5Shi"

time: 2.37 s (started: 2022-08-06 20:36:39 +00:00)


In [56]:
!git commit -a

hint: Waiting for your editor to close the file... 7[>4;2m="/kaggle/working/.git/COMMIT_EDITMSG" 24L, 693C▽  [>c]10;?]11;?# Please enter the commit message for your changes. Lines starting
# with '#' will be ignored, and an empty message aborts the commit.
#
# On branch master
#
# Initial commit
#
# Changes to be committed:
#       new file:   KeywordExtraction
#       new file:   Model_state_dict.pickle
#       new file:   Model_state_dict_1.pickle
#       new file:   __notebook_source__.ipynb
#       new file:   confusion-matrix.csv
#       new file:   confusion-matrix_1.csv
#       new file:   label_pred_dict.pkl
#       new file:   label_pred_dict_1.pkl
#       new file:   model.pickle
#       new file:   model_1.pickle
#       new file:   new-keywords-cate.csv
#       new file:   new-keywords-cate_1.csv
#       new file:   new_kws_dict.pkl
#       new file:   new_kws_dict_1.pkl1,0-1TopType  :qa  and press <Enter> to exit Vim1,0-1Toptime: 1h 35min 19s (started: 2022-08-06 

In [61]:
!git push --set-upstream https://github.com/Fan5Shi/KeywordExtraction.git

fatal: The current branch master has no upstream branch.
To push the current branch and set the remote as upstream, use

    git push --set-upstream https://github.com/Fan5Shi/KeywordExtraction.git master

time: 1.18 s (started: 2022-08-06 22:13:31 +00:00)


In [38]:
def get_precision(matrix):
    keys = matrix.sum(axis=0).keys()
    values = matrix.sum(axis=0).to_numpy()
    return [(keys[i], np.nan_to_num(matrix.iloc[i][i] / values[i])) for i in range(len(matrix)-1)]

def get_recall(matrix):
    keys = matrix.sum(axis=1).keys()
    values = matrix.sum(axis=1).to_numpy()
    return [(keys[i], np.nan_to_num(matrix.iloc[i][i] / values[i])) for i in range(len(matrix)-1)]

def get_f1score(matrix):
    precision = get_precision(matrix)
    recall = get_recall(matrix)
    return [(precision[i][0], 
             np.nan_to_num(2 * precision[i][1] * recall[i][1] / (precision[i][1] + recall[i][1])))
            for i in range(len(matrix)-1)]

def get_average_precision(matrix):
    precision = get_precision(matrix)
    return np.mean([precision[i][1] for i in range(len(precision))])

def get_average_recall(matrix):
    recall = get_recall(matrix)
    return np.mean([recall[i][1] for i in range(len(recall))])

def get_average_f1score(matrix):
    precision = get_average_precision(matrix)
    recall = get_average_recall(matrix)
    return 2 * precision * recall / (precision + recall)

time: 1.87 ms (started: 2022-08-06 20:11:35 +00:00)


In [40]:
get_average_precision(df), get_average_recall(df), get_average_f1score(df)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in long_scalars
  after removing the cwd from sys.path.


(0.5391806500168269, 0.5679443427203611, 0.5531888483959754)

time: 15.3 ms (started: 2022-08-06 20:11:52 +00:00)


In [13]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Highway(nn.Module):
    """
    Highway Network.
    """

    def __init__(self, size, num_layers=1, dropout=0.5):
        """
        :param size: size of linear layer (matches input size)
        :param num_layers: number of transform and gate layers
        :param dropout: dropout
        """
        super(Highway, self).__init__()
        self.size = size
        self.num_layers = num_layers
        self.transform = nn.ModuleList()  # list of transform layers
        self.gate = nn.ModuleList()  # list of gate layers
        self.dropout = nn.Dropout(p=dropout)

        for i in range(num_layers):
            transform = nn.Linear(size, size)
            gate = nn.Linear(size, size)
            self.transform.append(transform)
            self.gate.append(gate)

    def forward(self, x):
        """
        Forward propagation.

        :param x: input tensor
        :return: output tensor, with same dimensions as input tensor
        """
        transformed = nn.functional.relu(self.transform[0](x))  # transform input
        g = torch.sigmoid(self.gate[0](x))  # calculate how much of the transformed input to keep

        out = g * transformed + (1 - g) * x  # combine input and transformed input in this ratio

        # If there are additional layers
        for i in range(1, self.num_layers):
            out = self.dropout(out)
            transformed = nn.functional.relu(self.transform[i](out))
            g = torch.sigmoid(self.gate[i](out))

            out = g * transformed + (1 - g) * out

        return out


class CRF(nn.Module):
    """
    Conditional Random Field.
    """

    def __init__(self, hidden_dim, tagset_size):
        """
        :param hidden_dim: size of word RNN/BLSTM's output
        :param tagset_size: number of tags
        """
        super(CRF, self).__init__()
        self.tagset_size = tagset_size
        self.emission = nn.Linear(hidden_dim, self.tagset_size)
        self.transition = nn.Parameter(torch.Tensor(self.tagset_size, self.tagset_size))
        self.transition.data.zero_()

    def forward(self, feats):
        """
        Forward propagation.

        :param feats: output of word RNN/BLSTM, a tensor of dimensions (batch_size, timesteps, hidden_dim)
        :return: CRF scores, a tensor of dimensions (batch_size, timesteps, tagset_size, tagset_size)
        """
        self.batch_size = feats.size(0)
        self.timesteps = feats.size(1)

        emission_scores = self.emission(feats)  # (batch_size, timesteps, tagset_size)
        emission_scores = emission_scores.unsqueeze(2).expand(self.batch_size, self.timesteps, self.tagset_size,
                                                              self.tagset_size)  # (batch_size, timesteps, tagset_size, tagset_size)

        crf_scores = emission_scores + self.transition.unsqueeze(0).unsqueeze(
            0)  # (batch_size, timesteps, tagset_size, tagset_size)
        return crf_scores


class LM_LSTM_CRF(nn.Module):
    """
    The encompassing LM-LSTM-CRF model.
    """

    def __init__(self, tagset_size, charset_size, char_emb_dim, char_rnn_dim, char_rnn_layers, vocab_size,
                 lm_vocab_size, word_emb_dim, word_rnn_dim, word_rnn_layers, dropout, highway_layers=1):
        """
        :param tagset_size: number of tags
        :param charset_size: size of character vocabulary
        :param char_emb_dim: size of character embeddings
        :param char_rnn_dim: size of character RNNs/LSTMs
        :param char_rnn_layers: number of layers in character RNNs/LSTMs
        :param vocab_size: input vocabulary size
        :param lm_vocab_size: vocabulary size of language models (in-corpus words subject to word frequency threshold)
        :param word_emb_dim: size of word embeddings
        :param word_rnn_dim: size of word RNN/BLSTM
        :param word_rnn_layers:  number of layers in word RNNs/LSTMs
        :param dropout: dropout
        :param highway_layers: number of transform and gate layers
        """

        super(LM_LSTM_CRF, self).__init__()

        self.tagset_size = tagset_size  # this is the size of the output vocab of the tagging model

        self.charset_size = charset_size
        self.char_emb_dim = char_emb_dim
        self.char_rnn_dim = char_rnn_dim
        self.char_rnn_layers = char_rnn_layers

        self.wordset_size = vocab_size  # this is the size of the input vocab (embedding layer) of the tagging model
        self.lm_vocab_size = lm_vocab_size  # this is the size of the output vocab of the language model
        self.word_emb_dim = word_emb_dim
        self.word_rnn_dim = word_rnn_dim
        self.word_rnn_layers = word_rnn_layers

        self.highway_layers = highway_layers

        self.dropout = nn.Dropout(p=dropout)

        self.char_embeds = nn.Embedding(self.charset_size, self.char_emb_dim)  # character embedding layer
        self.forw_char_lstm = nn.LSTM(self.char_emb_dim, self.char_rnn_dim, num_layers=self.char_rnn_layers,
                                      bidirectional=False, dropout=dropout)  # forward character LSTM
        self.back_char_lstm = nn.LSTM(self.char_emb_dim, self.char_rnn_dim, num_layers=self.char_rnn_layers,
                                      bidirectional=False, dropout=dropout)  # backward character LSTM

        self.word_embeds = nn.Embedding(self.wordset_size, self.word_emb_dim)  # word embedding layer
        self.word_blstm = nn.LSTM(self.word_emb_dim + self.char_rnn_dim * 2, self.word_rnn_dim // 2,
                                  num_layers=self.word_rnn_layers, bidirectional=True, dropout=dropout)  # word BLSTM

        self.crf = CRF((self.word_rnn_dim // 2) * 2, self.tagset_size)  # conditional random field

        self.forw_lm_hw = Highway(self.char_rnn_dim, num_layers=self.highway_layers,
                                  dropout=dropout)  # highway to transform forward char LSTM output for the forward language model
        self.back_lm_hw = Highway(self.char_rnn_dim, num_layers=self.highway_layers,
                                  dropout=dropout)  # highway to transform backward char LSTM output for the backward language model
        self.subword_hw = Highway(2 * self.char_rnn_dim, num_layers=self.highway_layers,
                                  dropout=dropout)  # highway to transform combined forward and backward char LSTM outputs for use in the word BLSTM

        self.forw_lm_out = nn.Linear(self.char_rnn_dim,
                                     self.lm_vocab_size)  # linear layer to find vocabulary scores for the forward language model
        self.back_lm_out = nn.Linear(self.char_rnn_dim,
                                     self.lm_vocab_size)  # linear layer to find vocabulary scores for the backward language model

    def init_word_embeddings(self, embeddings):
        """
        Initialize embeddings with pre-trained embeddings.

        :param embeddings: pre-trained embeddings
        """
        self.word_embeds.weight = nn.Parameter(embeddings)

    def fine_tune_word_embeddings(self, fine_tune=False):
        """
        Fine-tune embedding layer? (Not fine-tuning only makes sense if using pre-trained embeddings).

        :param fine_tune: Fine-tune?
        """
        for p in self.word_embeds.parameters():
            p.requires_grad = fine_tune

    def forward(self, cmaps_f, cmaps_b, cmarkers_f, cmarkers_b, wmaps, tmaps, wmap_lengths, cmap_lengths):
        """
        Forward propagation.

        :param cmaps_f: padded encoded forward character sequences, a tensor of dimensions (batch_size, char_pad_len)
        :param cmaps_b: padded encoded backward character sequences, a tensor of dimensions (batch_size, char_pad_len)
        :param cmarkers_f: padded forward character markers, a tensor of dimensions (batch_size, word_pad_len)
        :param cmarkers_b: padded backward character markers, a tensor of dimensions (batch_size, word_pad_len)
        :param wmaps: padded encoded word sequences, a tensor of dimensions (batch_size, word_pad_len)
        :param tmaps: padded tag sequences, a tensor of dimensions (batch_size, word_pad_len)
        :param wmap_lengths: word sequence lengths, a tensor of dimensions (batch_size)
        :param cmap_lengths: character sequence lengths, a tensor of dimensions (batch_size, word_pad_len)
        """
        self.batch_size = cmaps_f.size(0)
        self.word_pad_len = wmaps.size(1)

        # Sort by decreasing true char. sequence length
        cmap_lengths, char_sort_ind = cmap_lengths.sort(dim=0, descending=True)
        cmaps_f = cmaps_f[char_sort_ind]
        cmaps_b = cmaps_b[char_sort_ind]
        cmarkers_f = cmarkers_f[char_sort_ind]
        cmarkers_b = cmarkers_b[char_sort_ind]
        wmaps = wmaps[char_sort_ind]
        tmaps = tmaps[char_sort_ind]
        wmap_lengths = wmap_lengths[char_sort_ind]

        # Embedding look-up for characters
        cf = self.char_embeds(cmaps_f)  # (batch_size, char_pad_len, char_emb_dim)
        cb = self.char_embeds(cmaps_b)

        # Dropout
        cf = self.dropout(cf)  # (batch_size, char_pad_len, char_emb_dim)
        cb = self.dropout(cb)

        # Pack padded sequence
        cf = pack_padded_sequence(cf, cmap_lengths.tolist(),
                                  batch_first=True)  # packed sequence of char_emb_dim, with real sequence lengths
        cb = pack_padded_sequence(cb, cmap_lengths.tolist(), batch_first=True)

        # LSTM
        cf, _ = self.forw_char_lstm(cf)  # packed sequence of char_rnn_dim, with real sequence lengths
        cb, _ = self.back_char_lstm(cb)

        # Unpack packed sequence
        cf, _ = pad_packed_sequence(cf, batch_first=True)  # (batch_size, max_char_len_in_batch, char_rnn_dim)
        cb, _ = pad_packed_sequence(cb, batch_first=True)

        # Sanity check
        assert cf.size(1) == max(cmap_lengths.tolist()) == list(cmap_lengths)[0]

        # Select RNN outputs only at marker points (spaces in the character sequence)
        cmarkers_f = cmarkers_f.unsqueeze(2).expand(self.batch_size, self.word_pad_len, self.char_rnn_dim)
        cmarkers_b = cmarkers_b.unsqueeze(2).expand(self.batch_size, self.word_pad_len, self.char_rnn_dim)
        cf_selected = torch.gather(cf, 1, cmarkers_f)  # (batch_size, word_pad_len, char_rnn_dim)
        cb_selected = torch.gather(cb, 1, cmarkers_b)

        # Only for co-training, not useful for tagging after model is trained
        if self.training:
            lm_f = self.forw_lm_hw(self.dropout(cf_selected))  # (batch_size, word_pad_len, char_rnn_dim)
            lm_b = self.back_lm_hw(self.dropout(cb_selected))
            lm_f_scores = self.forw_lm_out(self.dropout(lm_f))  # (batch_size, word_pad_len, lm_vocab_size)
            lm_b_scores = self.back_lm_out(self.dropout(lm_b))

        # Sort by decreasing true word sequence length
        wmap_lengths, word_sort_ind = wmap_lengths.sort(dim=0, descending=True)
        wmaps = wmaps[word_sort_ind]
        tmaps = tmaps[word_sort_ind]
        cf_selected = cf_selected[word_sort_ind]  # for language model
        cb_selected = cb_selected[word_sort_ind]
        if self.training:
            lm_f_scores = lm_f_scores[word_sort_ind]
            lm_b_scores = lm_b_scores[word_sort_ind]

        # Embedding look-up for words
        w = self.word_embeds(wmaps)  # (batch_size, word_pad_len, word_emb_dim)
        w = self.dropout(w)

        # Sub-word information at each word
        subword = self.subword_hw(self.dropout(
            torch.cat((cf_selected, cb_selected), dim=2)))  # (batch_size, word_pad_len, 2 * char_rnn_dim)
        subword = self.dropout(subword)

        # Concatenate word embeddings and sub-word features
        w = torch.cat((w, subword), dim=2)  # (batch_size, word_pad_len, word_emb_dim + 2 * char_rnn_dim)

        # Pack padded sequence
        w = pack_padded_sequence(w, list(wmap_lengths),
                                 batch_first=True)  # packed sequence of word_emb_dim + 2 * char_rnn_dim, with real sequence lengths

        # LSTM
        w, _ = self.word_blstm(w)  # packed sequence of word_rnn_dim, with real sequence lengths

        # Unpack packed sequence
        w, _ = pad_packed_sequence(w, batch_first=True)  # (batch_size, max_word_len_in_batch, word_rnn_dim)
        w = self.dropout(w)

        crf_scores = self.crf(w)  # (batch_size, max_word_len_in_batch, tagset_size, tagset_size)

        if self.training:
            return crf_scores, lm_f_scores, lm_b_scores, wmaps, tmaps, wmap_lengths, word_sort_ind, char_sort_ind
        else:
            return crf_scores, wmaps, tmaps, wmap_lengths, word_sort_ind, char_sort_ind  # sort inds to reorder, if req.


class ViterbiLoss(nn.Module):
    """
    Viterbi Loss.
    """

    def __init__(self, tag_map):
        """
        :param tag_map: tag map
        """
        super(ViterbiLoss, self).__init__()
        self.tagset_size = len(tag_map)
        self.start_tag = tag_map['<start>']
        self.end_tag = tag_map['<end>']

    def forward(self, scores, targets, lengths):
        """
        Forward propagation.

        :param scores: CRF scores
        :param targets: true tags indices in unrolled CRF scores
        :param lengths: word sequence lengths
        :return: viterbi loss
        """

        batch_size = scores.size(0)
        word_pad_len = scores.size(1)

        # Gold score

        targets = targets.unsqueeze(2)
        scores_at_targets = torch.gather(scores.view(batch_size, word_pad_len, -1), 2, targets).squeeze(
            2)  # (batch_size, word_pad_len)

        # Everything is already sorted by lengths
        scores_at_targets = pack_padded_sequence(scores_at_targets, lengths, batch_first=True)
        gold_score = scores_at_targets.data.sum()

        # All paths' scores

        # Create a tensor to hold accumulated sequence scores at each current tag
        scores_upto_t = torch.zeros(batch_size, self.tagset_size).to(device)

        for t in range(max(lengths)):
            batch_size_t = sum([l > t for l in lengths])  # effective batch size (sans pads) at this timestep
            if t == 0:
                scores_upto_t[:batch_size_t] = scores[:batch_size_t, t, self.start_tag, :]  # (batch_size, tagset_size)
            else:
                # We add scores at current timestep to scores accumulated up to previous timestep, and log-sum-exp
                # Remember, the cur_tag of the previous timestep is the prev_tag of this timestep
                # So, broadcast prev. timestep's cur_tag scores along cur. timestep's cur_tag dimension
                scores_upto_t[:batch_size_t] = log_sum_exp(
                    scores[:batch_size_t, t, :, :] + scores_upto_t[:batch_size_t].unsqueeze(2),
                    dim=1)  # (batch_size, tagset_size)

        # We only need the final accumulated scores at the <end> tag
        all_paths_scores = scores_upto_t[:, self.end_tag].sum()

        viterbi_loss = all_paths_scores - gold_score
        viterbi_loss = viterbi_loss / batch_size

        return viterbi_loss

time: 14.7 ms (started: 2022-09-09 08:42:53 +00:00)


In [12]:
from collections import Counter
import codecs
import itertools
from functools import reduce
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init
from torch.nn.utils.rnn import pack_padded_sequence


def read_words_tags(file, tag_ind, caseless=True):
    """
    Reads raw data in the CoNLL 2003 format and returns word and tag sequences.

    :param file: file with raw data in the CoNLL 2003 format
    :param tag_ind: column index of tag
    :param caseless: lowercase words?
    :return: word, tag sequences
    """
    with codecs.open(file, 'r', 'utf-8') as f:
        lines = f.readlines()
    words = []
    tags = []
    temp_w = []
    temp_t = []
    for line in lines:
        if not (line.isspace() or (len(line) > 10 and line[0:10] == '-DOCSTART-')):
            feats = line.rstrip('\n').split()
            temp_w.append(feats[0].lower() if caseless else feats[0])
            temp_t.append(feats[tag_ind])
        elif len(temp_w) > 0:
            assert len(temp_w) == len(temp_t)
            words.append(temp_w)
            tags.append(temp_t)
            temp_w = []
            temp_t = []
    # last sentence
    if len(temp_w) > 0:
        assert len(temp_w) == len(temp_t)
        words.append(temp_w)
        tags.append(temp_t)

    # Sanity check
    assert len(words) == len(tags)

    return words, tags


def create_maps(words, tags, min_word_freq=5, min_char_freq=1):
    """
    Creates word, char, tag maps.

    :param words: word sequences
    :param tags: tag sequences
    :param min_word_freq: words that occur fewer times than this threshold are binned as <unk>s
    :param min_char_freq: characters that occur fewer times than this threshold are binned as <unk>s
    :return: word, char, tag maps
    """
    word_freq = Counter()
    char_freq = Counter()
    tag_map = set()
    for w, t in zip(words, tags):
        word_freq.update(w)
        char_freq.update(list(reduce(lambda x, y: list(x) + [' '] + list(y), w)))
        tag_map.update(t)

    word_map = {k: v + 1 for v, k in enumerate([w for w in word_freq.keys() if word_freq[w] > min_word_freq])}
    char_map = {k: v + 1 for v, k in enumerate([c for c in char_freq.keys() if char_freq[c] > min_char_freq])}
    tag_map = {k: v + 1 for v, k in enumerate(tag_map)}

    word_map['<pad>'] = 0
    word_map['<end>'] = len(word_map)
    word_map['<unk>'] = len(word_map)
    char_map['<pad>'] = 0
    char_map['<end>'] = len(char_map)
    char_map['<unk>'] = len(char_map)
    tag_map['<pad>'] = 0
    tag_map['<start>'] = len(tag_map)
    tag_map['<end>'] = len(tag_map)

    return word_map, char_map, tag_map


def create_input_tensors(words, tags, word_map, char_map, tag_map):
    """
    Creates input tensors that will be used to create a PyTorch Dataset.

    :param words: word sequences
    :param tags: tag sequences
    :param word_map: word map
    :param char_map: character map
    :param tag_map: tag map
    :return: padded encoded words, padded encoded forward chars, padded encoded backward chars,
            padded forward character markers, padded backward character markers, padded encoded tags,
            word sequence lengths, char sequence lengths
    """
    # Encode sentences into word maps with <end> at the end
    # [['dunston', 'checks', 'in', '<end>']] -> [[4670, 4670, 185, 4669]]
    wmaps = list(map(lambda s: list(map(lambda w: word_map.get(w, word_map['<unk>']), s)) + [word_map['<end>']], words))

    # Forward and backward character streams
    # [['d', 'u', 'n', 's', 't', 'o', 'n', ' ', 'c', 'h', 'e', 'c', 'k', 's', ' ', 'i', 'n', ' ']]
    chars_f = list(map(lambda s: list(reduce(lambda x, y: list(x) + [' '] + list(y), s)) + [' '], words))
    # [['n', 'i', ' ', 's', 'k', 'c', 'e', 'h', 'c', ' ', 'n', 'o', 't', 's', 'n', 'u', 'd', ' ']]
    chars_b = list(
        map(lambda s: list(reversed([' '] + list(reduce(lambda x, y: list(x) + [' '] + list(y), s)))), words))

    # Encode streams into forward and backward character maps with <end> at the end
    # [[29, 2, 12, 8, 7, 14, 12, 3, 6, 18, 1, 6, 21, 8, 3, 17, 12, 3, 60]]
    cmaps_f = list(
        map(lambda s: list(map(lambda c: char_map.get(c, char_map['<unk>']), s)) + [char_map['<end>']], chars_f))
    # [[12, 17, 3, 8, 21, 6, 1, 18, 6, 3, 12, 14, 7, 8, 12, 2, 29, 3, 60]]
    cmaps_b = list(
        map(lambda s: list(map(lambda c: char_map.get(c, char_map['<unk>']), s)) + [char_map['<end>']], chars_b))

    # Positions of spaces and <end> character
    # Words are predicted or encoded at these places in the language and tagging models respectively
    # [[7, 14, 17, 18]] are points after '...dunston', '...checks', '...in', '...<end>' respectively
    cmarkers_f = list(map(lambda s: [ind for ind in range(len(s)) if s[ind] == char_map[' ']] + [len(s) - 1], cmaps_f))
    # Reverse the markers for the backward stream before adding <end>, so the words of the f and b markers coincide
    # i.e., [[17, 9, 2, 18]] are points after '...notsnud', '...skcehc', '...ni', '...<end>' respectively
    cmarkers_b = list(
        map(lambda s: list(reversed([ind for ind in range(len(s)) if s[ind] == char_map[' ']])) + [len(s) - 1],
            cmaps_b))

    # Encode tags into tag maps with <end> at the end
    tmaps = list(map(lambda s: list(map(lambda t: tag_map[t], s)) + [tag_map['<end>']], tags))
    # Since we're using CRF scores of size (prev_tags, cur_tags), find indices of target sequence in the unrolled scores
    # This will be row_index (i.e. prev_tag) * n_columns (i.e. tagset_size) + column_index (i.e. cur_tag)
    tmaps = list(map(lambda s: [tag_map['<start>'] * len(tag_map) + s[0]] + [s[i - 1] * len(tag_map) + s[i] for i in
                                                                             range(1, len(s))], tmaps))
    # Note - the actual tag indices can be recovered with tmaps % len(tag_map)

    # Pad, because need fixed length to be passed around by DataLoaders and other layers
    word_pad_len = max(list(map(lambda s: len(s), wmaps)))
    char_pad_len = max(list(map(lambda s: len(s), cmaps_f)))

    # Sanity check
    assert word_pad_len == max(list(map(lambda s: len(s), tmaps)))

    padded_wmaps = []
    padded_cmaps_f = []
    padded_cmaps_b = []
    padded_cmarkers_f = []
    padded_cmarkers_b = []
    padded_tmaps = []
    wmap_lengths = []
    cmap_lengths = []

    for w, cf, cb, cmf, cmb, t in zip(wmaps, cmaps_f, cmaps_b, cmarkers_f, cmarkers_b, tmaps):
        # Sanity  checks
        assert len(w) == len(cmf) == len(cmb) == len(t)
        assert len(cmaps_f) == len(cmaps_b)

        # Pad
        # A note -  it doesn't really matter what we pad with, as long as it's a valid index
        # i.e., we'll extract output at those pad points (to extract equal lengths), but never use them

        padded_wmaps.append(w + [word_map['<pad>']] * (word_pad_len - len(w)))
        padded_cmaps_f.append(cf + [char_map['<pad>']] * (char_pad_len - len(cf)))
        padded_cmaps_b.append(cb + [char_map['<pad>']] * (char_pad_len - len(cb)))

        # 0 is always a valid index to pad markers with (-1 is too but torch.gather has some issues with it)
        padded_cmarkers_f.append(cmf + [0] * (word_pad_len - len(w)))
        padded_cmarkers_b.append(cmb + [0] * (word_pad_len - len(w)))

        padded_tmaps.append(t + [tag_map['<pad>']] * (word_pad_len - len(t)))

        wmap_lengths.append(len(w))
        cmap_lengths.append(len(cf))

        # Sanity check
        assert len(padded_wmaps[-1]) == len(padded_tmaps[-1]) == len(padded_cmarkers_f[-1]) == len(
            padded_cmarkers_b[-1]) == word_pad_len
        assert len(padded_cmaps_f[-1]) == len(padded_cmaps_b[-1]) == char_pad_len

    padded_wmaps = torch.LongTensor(padded_wmaps)
    padded_cmaps_f = torch.LongTensor(padded_cmaps_f)
    padded_cmaps_b = torch.LongTensor(padded_cmaps_b)
    padded_cmarkers_f = torch.LongTensor(padded_cmarkers_f)
    padded_cmarkers_b = torch.LongTensor(padded_cmarkers_b)
    padded_tmaps = torch.LongTensor(padded_tmaps)
    wmap_lengths = torch.LongTensor(wmap_lengths)
    cmap_lengths = torch.LongTensor(cmap_lengths)

    return padded_wmaps, padded_cmaps_f, padded_cmaps_b, padded_cmarkers_f, padded_cmarkers_b, padded_tmaps, \
           wmap_lengths, cmap_lengths


def init_embedding(input_embedding):
    """
    Initialize embedding tensor with values from the uniform distribution.

    :param input_embedding: embedding tensor
    :return:
    """
    bias = np.sqrt(3.0 / input_embedding.size(1))
    nn.init.uniform_(input_embedding, -bias, bias)


def load_embeddings(emb_file, word_map, expand_vocab=True):
    """
    Load pre-trained embeddings for words in the word map.

    :param emb_file: file with pre-trained embeddings (in the GloVe format)
    :param word_map: word map
    :param expand_vocab: expand vocabulary of word map to vocabulary of pre-trained embeddings?
    :return: embeddings for words in word map, (possibly expanded) word map,
            number of words in word map that are in-corpus (subject to word frequency threshold)
    """
    with open(emb_file, 'r') as f:
        emb_len = len(f.readline().split(' ')) - 1

    print("Embedding length is %d." % emb_len)

    # Create tensor to hold embeddings for words that are in-corpus
    ic_embs = torch.FloatTensor(len(word_map), emb_len)
    init_embedding(ic_embs)

    if expand_vocab:
        print("You have elected to include embeddings that are out-of-corpus.")
        ooc_words = []
        ooc_embs = []
    else:
        print("You have elected NOT to include embeddings that are out-of-corpus.")

    # Read embedding file
    print("\nLoading embeddings...")
    for line in open(emb_file, 'r'):
        line = line.split(' ')

        emb_word = line[0]
        embedding = list(map(lambda t: float(t), filter(lambda n: n and not n.isspace(), line[1:])))

        if not expand_vocab and emb_word not in word_map:
            continue

        # If word is in train_vocab, store at the correct index (as in the word_map)
        if emb_word in word_map:
            ic_embs[word_map[emb_word]] = torch.FloatTensor(embedding)

        # If word is in dev or test vocab, store it and its embedding into lists
        elif expand_vocab:
            ooc_words.append(emb_word)
            ooc_embs.append(embedding)

    lm_vocab_size = len(word_map)  # keep track of lang. model's output vocab size (no out-of-corpus words)

    if expand_vocab:
        print("'word_map' is being updated accordingly.")
        for word in ooc_words:
            word_map[word] = len(word_map)
        ooc_embs = torch.FloatTensor(np.asarray(ooc_embs))
        embeddings = torch.cat([ic_embs, ooc_embs], 0)

    else:
        embeddings = ic_embs

    # Sanity check
    assert embeddings.size(0) == len(word_map)

    print("\nDone.\n Embedding vocabulary: %d\n Language Model vocabulary: %d.\n" % (len(word_map), lm_vocab_size))

    return embeddings, word_map, lm_vocab_size


def clip_gradient(optimizer, grad_clip):
    """
    Clip gradients computed during backpropagation to prevent gradient explosion.

    :param optimizer: optimized with the gradients to be clipped
    :param grad_clip: gradient clip value
    """
    for group in optimizer.param_groups:
        for param in group['params']:
            if param.grad is not None:
                param.grad.data.clamp_(-grad_clip, grad_clip)


def save_checkpoint(epoch, model, optimizer, val_f1, word_map, char_map, tag_map, lm_vocab_size, is_best):
    """
    Save model checkpoint.

    :param epoch: epoch number
    :param model: model
    :param optimizer: optimized
    :param val_f1: validation F1 score
    :param word_map: word map
    :param char_map: char map
    :param tag_map: tag map
    :param lm_vocab_size: number of words in-corpus, i.e. size of output vocabulary of linear model
    :param is_best: is this checkpoint the best so far?
    :return:
    """
    state = {'epoch': epoch,
             'f1': val_f1,
             'model': model,
             'optimizer': optimizer,
             'word_map': word_map,
             'tag_map': tag_map,
             'char_map': char_map,
             'lm_vocab_size': lm_vocab_size}
    filename = 'checkpoint_lm_lstm_crf.pth.tar'
    torch.save(state, filename)
    # If checkpoint is the best so far, create a copy to avoid being overwritten by a subsequent worse checkpoint
    if is_best:
        torch.save(state, 'BEST_' + filename)


class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, new_lr):
    """
    Shrinks learning rate by a specified factor.

    :param optimizer: optimizer whose learning rates must be decayed
    :param new_lr: new learning rate
    """

    print("\nDECAYING learning rate.")
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr
    print("The new learning rate is %f\n" % (optimizer.param_groups[0]['lr'],))


def log_sum_exp(tensor, dim):
    """
    Calculates the log-sum-exponent of a tensor's dimension in a numerically stable way.

    :param tensor: tensor
    :param dim: dimension to calculate log-sum-exp of
    :return: log-sum-exp
    """
    m, _ = torch.max(tensor, dim)
    m_expanded = m.unsqueeze(dim).expand_as(tensor)
    return m + torch.log(torch.sum(torch.exp(tensor - m_expanded), dim))

time: 13.4 ms (started: 2022-09-09 08:42:53 +00:00)


In [11]:
import torch


class ViterbiDecoder():
    """
    Viterbi Decoder.
    """

    def __init__(self, tag_map):
        """
        :param tag_map: tag map
        """
        self.tagset_size = len(tag_map)
        self.start_tag = tag_map['<start>']
        self.end_tag = tag_map['<end>']

    def decode(self, scores, lengths):
        """
        :param scores: CRF scores
        :param lengths: word sequence lengths
        :return: decoded sequences
        """
        batch_size = scores.size(0)
        word_pad_len = scores.size(1)

        # Create a tensor to hold accumulated sequence scores at each current tag
        scores_upto_t = torch.zeros(batch_size, self.tagset_size)

        # Create a tensor to hold back-pointers
        # i.e., indices of the previous_tag that corresponds to maximum accumulated score at current tag
        # Let pads be the <end> tag index, since that was the last tag in the decoded sequence
        backpointers = torch.ones((batch_size, max(lengths), self.tagset_size), dtype=torch.long) * self.end_tag

        for t in range(max(lengths)):
            batch_size_t = sum([l > t for l in lengths])  # effective batch size (sans pads) at this timestep
            if t == 0:
                scores_upto_t[:batch_size_t] = scores[:batch_size_t, t, self.start_tag, :]  # (batch_size, tagset_size)
                backpointers[:batch_size_t, t, :] = torch.ones((batch_size_t, self.tagset_size),
                                                               dtype=torch.long) * self.start_tag
            else:
                # We add scores at current timestep to scores accumulated up to previous timestep, and
                # choose the previous timestep that corresponds to the max. accumulated score for each current timestep
                scores_upto_t[:batch_size_t], backpointers[:batch_size_t, t, :] = torch.max(
                    scores[:batch_size_t, t, :, :] + scores_upto_t[:batch_size_t].unsqueeze(2),
                    dim=1)  # (batch_size, tagset_size)

        # Decode/trace best path backwards
        decoded = torch.zeros((batch_size, backpointers.size(1)), dtype=torch.long)
        pointer = torch.ones((batch_size, 1),
                             dtype=torch.long) * self.end_tag  # the pointers at the ends are all <end> tags

        for t in list(reversed(range(backpointers.size(1)))):
            decoded[:, t] = torch.gather(backpointers[:, t, :], 1, pointer).squeeze(1)
            pointer = decoded[:, t].unsqueeze(1)  # (batch_size, 1)

        # Sanity check
        assert torch.equal(decoded[:, 0], torch.ones((batch_size), dtype=torch.long) * self.start_tag)

        # Remove the <starts> at the beginning, and append with <ends> (to compare to targets, if any)
        decoded = torch.cat([decoded[:, 1:], torch.ones((batch_size, 1), dtype=torch.long) * self.start_tag],
                            dim=1)

        return decoded

time: 4.67 ms (started: 2022-09-09 08:42:53 +00:00)


In [10]:
from torch.utils.data import Dataset
import torch


class WCDataset(Dataset):
    """
    PyTorch Dataset for the LM-LSTM-CRF model. To be used by a PyTorch DataLoader to feed batches to the model.
    """

    def __init__(self, wmaps, cmaps_f, cmaps_b, cmarkers_f, cmarkers_b, tmaps, wmap_lengths, cmap_lengths):
        """
        :param wmaps: padded encoded word sequences
        :param cmaps_f: padded encoded forward character sequences
        :param cmaps_b: padded encoded backward character sequences
        :param cmarkers_f: padded forward character markers
        :param cmarkers_b: padded backward character markers
        :param tmaps: padded encoded tag sequences (indices in unrolled CRF scores)
        :param wmap_lengths: word sequence lengths
        :param cmap_lengths: character sequence lengths
        """
        self.wmaps = wmaps
        self.cmaps_f = cmaps_f
        self.cmaps_b = cmaps_b
        self.cmarkers_f = cmarkers_f
        self.cmarkers_b = cmarkers_b
        self.tmaps = tmaps
        self.wmap_lengths = wmap_lengths
        self.cmap_lengths = cmap_lengths

        self.data_size = self.wmaps.size(0)

    def __getitem__(self, i):
        return self.wmaps[i], self.cmaps_f[i], self.cmaps_b[i], self.cmarkers_f[i], self.cmarkers_b[i], self.tmaps[i], \
               self.wmap_lengths[i], self.cmap_lengths[i]

    def __len__(self):
        return self.data_size

time: 4.8 ms (started: 2022-09-09 08:42:53 +00:00)


### Other models

In [1]:
# LSTM
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from torch.nn import utils as nn_utils


class LSTMEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTMEncoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, batch_first=True,
                            num_layers=1, bidirectional=True)
        self.dropout = nn.Dropout(0.5)
    

    def init_hidden(self, batch_size):
        return (torch.randn(2, batch_size, self.hidden_dim // 2).to(device),
                torch.randn(2, batch_size, self.hidden_dim // 2).to(device))
    
    def forward(self, sentences, seq_lens):
        '''
        Args:
            sentences: batch_size*word_num, long tensor
            seq_lens: batch_size, long tensor
        Return:
            hidden states for each sequence of tokens
        '''
        batch_size, word_num = sentences.size()
        
        self.hidden = self.init_hidden(batch_size)
#         print(sentences.device)
        embeds = self.word_embeds(sentences)#batch_size*word_num*emb_dim
        embeds = self.dropout(embeds)
        
        #Sorting according to the lengths
        perm_seq_lens, perm_idx = seq_lens.sort(0, descending=True)
        _, desorted_perm_idx = torch.sort(perm_idx, descending=False)
        embeds = embeds[perm_idx]
        pack_embeds = nn_utils.rnn.pack_padded_sequence(embeds, 
                                                        perm_seq_lens.to('cpu'), batch_first=True)
        
        lstm_out, self.hidden = self.lstm(pack_embeds, self.hidden)
        lstm_out, _ = nn_utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)
        #Restore the order of sentences
        return lstm_out[desorted_perm_idx]

1. Word embedding
2. Training

In [21]:
import torchtext
def tokenize_example(example, max_length=300):
    return example.split(' ')[:max_length]
def split_tags(example, max_length=300):
    return example.split(',')[:max_length]

data2['tokens'] = data2['sentence'].map(tokenize_example)
data2['labels'] = data2['word_labels2'].map(split_tags)

train_size = 0.8
train_dataset2 = data2.sample(frac=train_size,random_state=200)
test_dataset2 = data2.drop(train_dataset2.index).reset_index(drop=True)
train_dataset2 = train_dataset2.reset_index(drop=True)

min_freq = 3
special_tokens = ['<unk>', '<pad>']

vocab = torchtext.vocab.build_vocab_from_iterator(train_dataset2['tokens'],
                                                  min_freq=min_freq,
                                                  specials=special_tokens)

In [22]:
unk_index = vocab['<unk>']
pad_index = vocab['<pad>']
vocab.set_default_index(unk_index)

In [13]:
vectors = torchtext.vocab.FastText()

.vector_cache/wiki.en.vec: 6.60GB [03:23, 32.4MB/s]                                
100%|██████████| 2519370/2519370 [05:20<00:00, 7849.24it/s]


time: 9min 41s (started: 2022-07-27 17:34:07 +00:00)


In [14]:
from torch.nn.utils.rnn import pad_sequence
def sent2ids(text):
    return [vocab[w] for w in text]

def labels2ids(labels):
    return [labels_to_ids2[w] for w in labels]


def data_generator(sents, labels, batch_size=32, is_training=True, index=0):
    if is_training:
        select_indices = np.random.choice(len(sents), batch_size, replace=False)
    else:
        start = index
        end = min(start + batch_size, len(sents)) 
        select_indices = list(range(start, end))
    #select_indices = list(range(batch_size))
    batch_sents = np.array(sents)[select_indices]
    batch_labels = np.array(labels)[select_indices]
    
    batch_sents = list(map(sent2ids, batch_sents))
    batch_labels = list(map(labels2ids, batch_labels))
    
    seq_lens = [len(s) for s in batch_sents]
    seq_lens = torch.LongTensor(seq_lens)
    max_len = max(seq_lens)
    
    batch_sents = [torch.LongTensor(s) for s in batch_sents]
    
    
    batch_sents = pad_sequence(batch_sents, batch_first=True)
    
    if not is_training:
        return batch_sents, batch_labels, seq_lens, end
    batch_labels = [torch.LongTensor(s) for s in batch_labels]
    batch_labels = pad_sequence(batch_labels, batch_first=True)

    return batch_sents, batch_labels, seq_lens

time: 2.07 ms (started: 2022-07-27 17:45:30 +00:00)


In [15]:
# deal with the input format
train_sent_words = train_dataset2['tokens']
train_sent_tags = train_dataset2['labels']
batch_sents, batch_label, seq_lens = data_generator(train_sent_words, train_sent_tags, batch_size=32)

time: 23.1 ms (started: 2022-07-27 17:45:30 +00:00)


In [16]:
class LinearChainCRF(nn.Module):
    def __init__(self, label_size):
        super(LinearChainCRF, self).__init__()
        
        self.label_size = label_size
        self.start_label = label_size - 1
        self.stop_label = label_size - 2
        self.transitions = nn.Parameter(
            torch.randn(self.label_size, self.label_size))
        self.transitions.data[:, self.start_label] = -10000
        self.transitions.data[self.stop_label] = -10000
     
    def forward_alg(self, features):
        '''
        Forward algorithm
        Arg:
            features: features of a sentence, sent_len*label_size, tensor
        Return:
            alpha: the log sum of 
        '''
        #Complete the code
        init_alphas = torch.full((1, self.label_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0, self.start_label] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas.to(device)
        # Iterate through the sentence
        for feat in features:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.label_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.label_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[:, next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[:, self.stop_label]
        alpha = log_sum_exp(terminal_var)
        return alpha
    
    def sentence_scorer(self, features, labels):
        '''
        Score a sentence given its label sequence
        Args:
            features: features of a sentence, sent_len*label_size, tensor
            labels: a sequence of labels, sent_len, tensor
        Return:
            score: the score of the labeled sentence, a scalar
        '''
        #Complete the code
        score = torch.zeros(1).to(device)
        labels = torch.cat([torch.tensor([self.start_label], dtype=torch.long).to(device), labels])
        for i, feat in enumerate(features):
#             print(labels[i+1])
            score = score + \
                self.transitions[labels[i], labels[i + 1]] + feat[labels[i + 1]]
        score = score + self.transitions[labels[-1], torch.tensor([self.stop_label], dtype=torch.long).to(device)]
        return score
    
    def viterbi_decoder(self, features):
        '''
        Viterbi decoder
        Arg:
            features: features of a sentence, sent_len*label_size, tensor
        Return:
            best_path_score: best path score, a scalar
            best_path: best path, a list
        '''
        #Complete the code
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.label_size), -10000.)
        init_vvars[0][self.start_label] = 0
        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars.to(device)
        for feat in features:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.label_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[:, next_tag].view(1, -1)
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[:, self.stop_label].view(1, -1)
        best_tag_id = argmax(terminal_var)
        best_path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.start_label  # Sanity check
        best_path.reverse()
        return best_path_score, best_path
    

def argmax(x):
    # return the argmax as a python int
    _, idx = torch.max(x, 1)
    return idx.item()

    
def log_sum_exp(x):
    #return log sum exp of a tensor
    m = torch.max(x, -1)[0]
    return m + torch.log(torch.sum(torch.exp(x - m.unsqueeze(-1)), -1))

time: 4.47 ms (started: 2022-07-27 17:45:31 +00:00)


In [17]:
class SequenceLabeling(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size, path):
        super(SequenceLabeling, self).__init__()
        self.crf = LinearChainCRF(label_size)
        self.bilstm = LSTMEncoder(vocab_size, embedding_dim, hidden_dim)
        self.hidden2label_space = nn.Linear(hidden_dim, label_size)
        self.load_pretrained_emb(path)
        
        
    def forward(self, sentences, seq_lens, sent_tags):
        '''
        Compute the loss
        Args:
            sentences: batch_size*word_num, long tensor
            seq_lens: batch_size, long tensor
            sent_tags: batch_size*word_num, long tensor
        Return:
            loss: the average loss of a batch
        '''
        sentences = sentences.to(device)
        seq_lens = seq_lens.to(device)
        sent_tags = sent_tags.to(device)
        hiddens = self.bilstm(sentences, seq_lens)
        #batch_size*word_num*tag_size
        batch_feats = self.hidden2label_space(hiddens)
        neg_likelihoods = []
        for i, feats in enumerate(batch_feats):
            length = seq_lens[i]
            tags = sent_tags[i]
#             print(feats.device, tags.device)
            gold_score = self.crf.sentence_scorer(feats[:length], tags[:length])
            forward_score = self.crf.forward_alg(feats[:length])
            neg_likelihoods.append(forward_score-gold_score)
        loss = torch.stack(neg_likelihoods).mean()
        return loss

    def decoder(self, sentences, seq_lens):
        '''
        Viterbi decoder
        Args:
            sentences: batch_size*word_num, long tensor
            seq_lens: batch_size, long tensor
        Return:
            pred_path: batch_size, list
        '''
        sentences = sentences.to(device)
        seq_lens = seq_lens.to(device)
        hiddens = self.bilstm(sentences, seq_lens)
        #batch_size*word_num*tag_size
        batch_feats = self.hidden2label_space(hiddens)
        preds = []
        for i, feats in enumerate(batch_feats):
            length = seq_lens[i]
            _, pred = self.crf.viterbi_decoder(feats[:length])
            preds.append(pred)
        return preds
    
    def load_pretrained_emb(self, vectors):
        '''
        Load pre-trained word embeddings from the path
        Arg:
            path: the binary file of local Glove embeddings
        '''
#         with open(path, 'rb') as f:
# pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())
# model.embedding.weight.data = pretrained_embedding
        
        print("Vectors with shape {}".format(vectors.shape))
        print(f"Size of bi-LSTM {self.bilstm.word_embeds.weight.size()}")
        assert vectors.shape == self.bilstm.word_embeds.weight.size()
        self.bilstm.word_embeds.weight.data = vectors
        self.bilstm.word_embeds.weight.requires_grad = False
        print('embeddings loaded')

time: 2.69 ms (started: 2022-07-27 17:45:31 +00:00)


In [18]:
from torch import optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())
model = SequenceLabeling(len(vocab), 300, 64, len(labels_to_ids2), pretrained_embedding)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

cuda
Vectors with shape torch.Size([32731, 300])
Size of bi-LSTM torch.Size([32731, 300])
embeddings loaded
time: 1.25 s (started: 2022-07-27 17:45:31 +00:00)


In [1]:
from time import time
epoch = 3
loop_num = int(len(train_sent_words)/32)
print(f"The loop numbers: {loop_num}")
for i in range(epoch):
    start = time()
    model.train()
    for j in range(loop_num):
        optimizer.zero_grad()
        batch_sents, batch_tags, seq_lens = data_generator(train_sent_words,train_sent_tags)
        loss = model(batch_sents, seq_lens, batch_tags)
        loss.backward()
        optimizer.step()
        if j % 20 == 0:
            print(f'Loss: {loss.item()} === Cost time per step: {time() - start}')

NameError: name 'train_sent_words' is not defined

In [37]:
# model_scripted = torch.jit.script(model) # Export to TorchScript
# model_scripted.save('model_scripted.pt') # Save

time: 11.9 ms (started: 2022-07-27 10:00:07 +00:00)


In [38]:
# torch.save(model, "Model_entire.pickle")
# # model.state_dict

time: 10.5 ms (started: 2022-07-27 10:00:08 +00:00)


Evaluation

In [22]:
dev_sent_words = test_dataset2['tokens']
dev_sent_tags = test_dataset2['labels']
model.eval()
index = 0
pred_label_list = []
while index < len(dev_sent_words):
    batch_sents, batch_tags, seq_lens, index = data_generator(dev_sent_words, 
                                                              dev_sent_tags, batch_size=32, 
                                                              is_training=False, index=index)
    pred_labels = model.decoder(batch_sents, seq_lens)
    for label_seq in pred_labels:
        pred_labels = [ids_to_labels2[t] for t in label_seq]
        pred_label_list.append(pred_labels)

time: 8.05 s (started: 2022-07-27 12:21:49 +00:00)


In [23]:
predictions = [p for pl in pred_label_list for p in pl]
labels = [d for dev in dev_sent_tags for d in dev]
words = [d for dev in dev_sent_words for d in dev]
len(predictions), len(labels), len(words)

(9685, 9685, 9685)

time: 14.6 ms (started: 2022-07-27 12:22:02 +00:00)


In [24]:
# newly generated words
def lengthOfTokens(pair):
    return len(pair.split())

def countKeywords(test_dataset, model):
    kws_pairs = []
    for tmp_num in range(len(test_dataset)):
        sentence = test_dataset["sentence"].iloc[tmp_num]

        inputs = tokenizer(sentence.split(),
                            is_split_into_words=True, 
                            return_offsets_mapping=True, 
                            padding='max_length', 
                            truncation=True, 
                            max_length=MAX_LEN,
                            return_tensors="pt")

        # move to gpu
        ids = inputs["input_ids"].to(device)
        mask = inputs["attention_mask"].to(device)
        # forward pass
        outputs = model(ids, attention_mask=mask)
        logits = outputs.logits

        active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

        tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
        token_predictions = [ids_to_labels2[i] for i in flattened_predictions.cpu().numpy()]
        wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

        prediction = []
        for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
          #only predictions on first word pieces are important
          if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
          else:
            continue

        for i in zip(sentence.split(), prediction, test_dataset["word_labels2"].iloc[tmp_num].split(',')):
            kws_pairs.append(i)

    predict_kws = []
    tmp_wd = []
    tmp_pred, tmp_gold = '', ''
    for word, pred, gold in kws_pairs:
        if pred != 'O':
            if len(tmp_wd) == 0: 
                tmp_pred = pred
            else:
                if tmp_pred != pred:
                    predict_kws.append((' '.join(tmp_wd), tmp_pred))
                    tmp_wd = []
                    tmp_pred = pred
            tmp_wd.append(word)
        else:
            if len(tmp_wd) > 0:
                predict_kws.append((' '.join(tmp_wd), tmp_pred))
                tmp_wd = []
                tmp_pred = pred

    # check whether there are new keywords
    gold_dict = build_gold_dict()
    new_kws, kw_tags = [], []
    gold_lists = list(gold_dict.keys())
    gold_lists = [g.lower() for g in gold_lists]
    for word, label in predict_kws:
        if not word.lower() in gold_lists:
            new_kws.append(word.lower())
            kw_tags.append(tag2cat[label])
    new_keywords = list(zip(new_kws, kw_tags))

    new_kws_dict, new_kws_pos = {}, []
    for kw, cat in new_keywords:
        pos = pos_tag(kw.split())
        if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
            key = ' '.join([p for p, t in pos])
            tag = [t for p, t in pos]
            new_kws_pos.append(key)
            if not key in new_kws_dict.keys():
                new_kws_dict[key] = (tag, cat)

    new_kws = list(new_kws_dict.keys())
    cats = [value[1] for value in new_kws_dict.values()]

    df = pd.DataFrame({"keyword": new_kws, "category": cats})

    s = df['keyword'].apply(lengthOfTokens)
    s.sort_values()

    predict_kws = Counter(predict_kws)
    new_kws_pos = Counter(new_kws_pos)

    predict_kws = sorted(predict_kws.items(), key=lambda pair: pair[1], reverse=True)
    new_kws_pos = sorted(new_kws_pos.items(), key=lambda pair: pair[1], reverse=True)

    return predict_kws, new_kws_pos, df

time: 4.24 ms (started: 2022-07-27 12:22:14 +00:00)


In [30]:
# tokens = [t for tags in dev_sent_tags for t in tags]
# token_predictions = [p for pred in pred_label_list for p in pred]
# len(tokens), len(token_predictions)
kws_pairs = []
for i in zip(words, predictions, labels):
    kws_pairs.append(i)
    
predict_kws = []
tmp_wd = []
tmp_pred, tmp_gold = '', ''
for word, pred, gold in kws_pairs:
    if pred != 'O':
        if len(tmp_wd) == 0: 
            tmp_pred = pred
        else:
            if tmp_pred != pred:
                predict_kws.append((' '.join(tmp_wd), tmp_pred))
                tmp_wd = []
                tmp_pred = pred
        tmp_wd.append(word)
    else:
        if len(tmp_wd) > 0:
            predict_kws.append((' '.join(tmp_wd), tmp_pred))
            tmp_wd = []
            tmp_pred = pred

# check whether there are new keywords
gold_dict = build_gold_dict()
new_kws, kw_tags = [], []
gold_lists = list(gold_dict.keys())
gold_lists = [g.lower() for g in gold_lists]
for word, label in predict_kws:
    if not word.lower() in gold_lists:
        new_kws.append(word.lower())
        kw_tags.append(tag2cat[label])
new_keywords = list(zip(new_kws, kw_tags))

new_kws_dict, new_kws_pos = {}, []
for kw, cat in new_keywords:
    pos = pos_tag(kw.split())
    if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
        key = ' '.join([p for p, t in pos])
        tag = [t for p, t in pos]
        new_kws_pos.append(key)
        if not key in new_kws_dict.keys():
            new_kws_dict[key] = (tag, cat)

new_kws = list(new_kws_dict.keys())
cats = [value[1] for value in new_kws_dict.values()]

df = pd.DataFrame({"keyword": new_kws, "category": cats})

s = df['keyword'].apply(lengthOfTokens)
s.sort_values()

predict_kws = Counter(predict_kws)
new_kws_pos = Counter(new_kws_pos)

predict_kws = sorted(predict_kws.items(), key=lambda pair: pair[1], reverse=True)
new_kws_pos = sorted(new_kws_pos.items(), key=lambda pair: pair[1], reverse=True)

# predict_kws, new_kws_pos, df

# predict_kws, new_kws_pos, tmp_kws_df = countKeywords(test_dataset2, model)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  


time: 592 ms (started: 2022-07-27 12:25:07 +00:00)


In [32]:
new_kws_dict = {}
new_kw_df = pd.DataFrame({})
new_kws_dict['predicted_keywords_lstm'] = predict_kws
new_kws_dict['new_keywords_epoch_lstm'] = new_kws_pos

new_kw_df = pd.concat([new_kw_df, df])
new_kw_df.reset_index(drop=True)

,keyword,category
0,aerospace,Change in management
1,cost,Change in management
2,intel,Business Model
3,core,Change in management
4,property,Change in management
...,...,...
109,america,Business Model
110,russia,Change in management
111,subsidiaries,Business Model
112,future,Change in management


time: 13.2 ms (started: 2022-07-27 12:27:08 +00:00)


In [95]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

def get_distribution(inputs, name):
    true_res = {}
    for k, v in tag2cat.items():
        indexlist = get_index_positions(inputs, k)
        left, right = 0, 0
        new_labels = []
        for i in indexlist:
            if left == 0:
                left = i
                right = i
                continue
            if right+1 == i:
                right = i
            else:
                new_labels.append((left, right))
                left = i
                right = i
        true_res[v] = new_labels
    print(f"The distribution of {name} is:")
    for k,v in true_res.items():
        print(f"There are {len(v)} occurances in {k}")
    return true_res

def get_confusion_matrix(true_labels, true_predictions):
    # strict mode
    tp_dict = {}
    for true_k, true_v in true_labels.items():
        true_total = len(true_v)
        tmp_dict = {}
        tmp_total = 0
        for pred_k, pred_v in true_predictions.items():
            num = 0
            for v in pred_v:
                if v in true_v:
                    num += 1
            tmp_dict[pred_k] = num
            tmp_total += num
        tmp_dict["O"] = true_total - tmp_total
        tp_dict[true_k] = tmp_dict
    tp_dict
    tmp_dict = {}
    for pred_k, pred_v in true_predictions.items():
        true_total = len(pred_v)
        num = 0
        for tp_k, tp_v in tp_dict.items():
            num += tp_v[pred_k]
        tmp_dict[pred_k] = true_total - num
    tmp_dict["O"] = 0
    tp_dict["O"] = tmp_dict
    return pd.DataFrame(tp_dict).T

time: 3.05 ms (started: 2022-08-01 15:58:26 +00:00)


In [34]:
true_labels = get_distribution(labels, "labels")
true_predictions = get_distribution(predictions, "predictions")
df = get_confusion_matrix(true_labels, true_predictions)
df

The distribution of labels is:
There are 16 occurances in Sustainability preoccupations
There are 14 occurances in Digital transformation
There are 1 occurances in Change in management
There are 27 occurances in Innovation activities
There are 10 occurances in Business Model
There are 0 occurances in Corporate social responsibility ou CSR
There are 0 occurances in marco-label
The distribution of predictions is:
There are 0 occurances in Sustainability preoccupations
There are 0 occurances in Digital transformation
There are 198 occurances in Change in management
There are 23 occurances in Innovation activities
There are 139 occurances in Business Model
There are 0 occurances in Corporate social responsibility ou CSR
There are 0 occurances in marco-label


,Sustainability preoccupations,Digital transformation,Change in management,Innovation activities,Business Model,Corporate social responsibility ou CSR,marco-label,O
Sustainability preoccupations,0,0,0,0,0,0,0,16
Digital transformation,0,0,0,0,0,0,0,14
Change in management,0,0,0,0,0,0,0,1
Innovation activities,0,0,0,0,0,0,0,27
Business Model,0,0,0,0,0,0,0,10
Corporate social responsibility ou CSR,0,0,0,0,0,0,0,0
marco-label,0,0,0,0,0,0,0,0
O,0,0,198,23,139,0,0,0


time: 15.3 ms (started: 2022-07-27 12:28:08 +00:00)


In [82]:
def write_embeddings(path, embeddings, vocab):
    
    with open(path, 'w') as f:
        for i, embedding in enumerate(tqdm(embeddings)):
            word = vocab.get_itos()[i]
            #skip words with unicode symbols
            if len(word) != len(word.encode()):
                continue
            vector = ' '.join([str(i) for i in embedding.tolist()])
            f.write(f'{word} {vector}\n')

write_embeddings('bilstm_trained_embeddings.txt', 
                 model.bilstm.word_embeds.weight.data, 
                 vocab)

  0%|          | 0/26746 [00:00<?, ?it/s]

time: 1min 6s (started: 2022-07-25 15:43:49 +00:00)


### CNN

### Previous Experiments

In [ ]:
datasetdf = pd.read_csv("../input/annual-report/dataset_ap_07_18.csv")
datasetdf = datasetdf.drop_duplicates(subset=['Company', 'Year', 'Text_para', 'Text_block', 'Catogory', 'Keyword'], keep='first', ignore_index=True)
datasetdf.drop(datasetdf.columns[datasetdf.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

def remove_back_slash_n(text):
    return text.replace('\n', ' ')
datasetdf['Text_para'] = datasetdf['Text_para'].apply(remove_back_slash_n)

In [ ]:
# fixed window
fwdf = pd.read_csv("../input/ftoken/dataset_07_07_ftoken.csv")
fwdf = fwdf.drop_duplicates(subset=['Company', 'Sector', 'Text_para', 'Nb_company', 'Text_block', 'Catogory', 'Keyword'], keep='first', ignore_index=True)
fwdf.drop(fwdf.columns[fwdf.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
print(len(fwdf))
fwdf = fwdf.dropna()
print(len(fwdf))
def remove_back_slash_n(text):
    return text.replace('\n', ' ')
fwdf['Text_para'] = fwdf['Text_para'].apply(remove_back_slash_n)

In [8]:
tag_dict = {}
tag_dict['Sustainability preoccupations'] = 'I-sus'
tag_dict['Digital transformation'] = 'I-dig'
tag_dict['Change in management'] = 'I-mag'
tag_dict['Innovation activities'] = 'I-inn'
tag_dict['Business Model'] = 'I-bus'
tag_dict['Corporate social responsibility ou CSR'] = 'I-cor'
tag_dict['marco-label'] = 'I-mar'
tag2cat = {v: k for k, v in tag_dict.items()}

labels_to_ids2 = {'O':0, 'I-sus':1, 'I-dig':2, 'I-mag':3, 'I-inn':4, 'I-bus':5, 'I-cor':6, 'I-mar':7}
ids_to_labels2 = {v: k for k, v in labels_to_ids2.items()}

gold_dict = build_gold_dict()

In [ ]:
def DatasetBuilder_tr(datasetdf):
    TAGS2, TEXTS2 = [], []
    for m in range(len(datasetdf)):
        ts = datasetdf['Text_para'].iloc[m].split()
#         print(datasetdf['Keyword'].iloc[m])
        ks = datasetdf['Keyword'].iloc[m].split(',')
        # deal with the special case
        texts = []
        for t in ts:
            if t == '4.0' or t == 'R' or t == 'D' or t == '&':
                texts.append(t)
            else:
                # text = re.findall(r"([\w']+(?:\S-\S)?[\w'])+", t)
                text = re.findall(r"([\w']+[-]?[\w']+)+", t)
                texts.extend(text)
        # deal with the special case
        keywords = []
        for k in ks:
            if k == 'Maintenance 4.0' or k == 'R & D' or k == 'R & D teams' or k == 'revolution 4.0':
                keywords.append(k.split())
            else:
                keyword = re.findall(r"([\w']+[-]?[\w']+)+", k)
                keywords.append(keyword)
        tags = ['O'] * len(texts)
        # cases: for each of the keyword in each block
        # 1. only one keyword in the block, and no marco lable overlapping
        # 2. more keyword in the block, and no marco label overlapping
        # 3. only one keyword in the block, but there's marco label overlapping
        # 4. more keyword in the block, and maroc label overlapping exists
        for kw in keywords:
            tt = tag_dict[gold_dict[' '.join(kw)][0]]
            length_kw = len(kw)
            item = kw[0]

            if length_kw == 1:
                start = 0
                end = len(texts)
                while True:
                    try: 
                        index = texts.index(item, start, end)
                        tags[index] = tt
                        start = index+1
                    except:
                        break
            
            if length_kw == 2:
                start = 0
                end = len(texts)
                while True:
                    try: 
                        index = texts.index(item, start, end)
                        start = index+1
                        if texts[index+1] == kw[1]:
                            if (tags[index] != 'O' and tags[index] != tt) or (tags[index+1] != 'O' and tags[index+1] != tt):
                                tt = 'I-mar'
                            for i in range(index, index+length_kw):
                                tags[i] = tt
                    except:
                        break

            if length_kw >= 3:
                start = 0
                end = len(texts)
                while True:
                    try: 
                        index = texts.index(item, start, end)
                        start = index+1
                        if texts[index+1] == kw[1] and texts[index+2] == kw[2]:
                            if (tags[index] != 'O' and tags[index] != tt) or (tags[index+1] != 'O' and tags[index+1] != tt) or (tags[index+2] != 'O' and tags[index+2] != tt):
                                tt = 'I-mar'
                            for i in range(index, index+length_kw):
                                tags[i] = tt
                    except:
                        break

        TAGS2.append(tags)
        TEXTS2.append(texts)

    datasetdf['word_labels2'] = [','.join(bt) for bt in TAGS2]
    datasetdf['sentence'] = [' '.join(tx) for tx in TEXTS2]

    return datasetdf[["sentence", "word_labels2"]].drop_duplicates().reset_index(drop=True)

In [ ]:
data2 = DatasetBuilder_tr(datasetdf)

In [ ]:
data3 = DatasetBuilder_tr(fwdf)

In [ ]:
class dataset2(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.sentence[index].strip().split()  
        word_labels = self.data.word_labels2[index].split(",") 

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_split_into_words=True, 
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids2[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train2(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model2.train()
    
    for idx, batch in enumerate(training_loader2):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        results = model2(input_ids=ids, attention_mask=mask, labels=labels)
        loss = results.loss
        tr_logits = results.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
#         if idx % 100==0:
#             loss_step = tr_loss/nb_tr_steps
#             print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model2.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model2.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer2.zero_grad()
        loss.backward()
        optimizer2.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    return epoch_loss, tr_accuracy

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels, eval_probability = [], [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            results = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = results.loss
            eval_logits = results.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
#             if idx % 100==0:
#                 loss_step = eval_loss/nb_eval_steps
#                 print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            
            flattened_probability = F.softmax(active_logits, dim=1)
            
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            probability = []
            for i, act in enumerate(active_accuracy):
                if act:
                    probability.append(flattened_probability[i])
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            eval_probability.extend(probability)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

#     labels = [ids_to_labels2[id.item()] for id in eval_labels]
#     predictions = [ids_to_labels2[id.item()] for id in eval_preds]
    labels = eval_labels
    predictions = eval_preds
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_loss, eval_accuracy, eval_probability

In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 4
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

print("ANNUAL REPORT Dataset: {}".format(data2.shape))

data_set2 = dataset2(data2, tokenizer, MAX_LEN)

train_params2 = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

data_loader2 = DataLoader(data_set2, **train_params2)

model2 = BertForTokenClassification.from_pretrained("../input/bert-base-model")
# model2 = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(labels_to_ids2))
# model2 = RobertaForTokenClassification.from_pretrained('roberta-base', num_labels=len(labels_to_ids2))
model2.to(device)

optimizer2 = torch.optim.Adam(params=model2.parameters(), lr=LEARNING_RATE)

labels, predictions, eval_loss, eval_accuracy, eval_probabilty = valid(model2, data_loader2) 

In [ ]:
def lengthOfTokens(pair):
        return len(pair.split())

def countKeywords(test_dataset, model):
    kws_pairs = []
    for tmp_num in range(len(test_dataset)):
        sentence = test_dataset["sentence"].iloc[tmp_num]

        inputs = tokenizer(sentence.split(),
                            is_split_into_words=True, 
                            return_offsets_mapping=True, 
                            padding='max_length', 
                            truncation=True, 
                            max_length=MAX_LEN,
                            return_tensors="pt")

        # move to gpu
        ids = inputs["input_ids"].to(device)
        mask = inputs["attention_mask"].to(device)
        # forward pass
        outputs = model(ids, attention_mask=mask)
        logits = outputs.logits

        active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

        tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
        token_predictions = [ids_to_labels2[i] for i in flattened_predictions.cpu().numpy()]
        wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

        prediction = []
        for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
          #only predictions on first word pieces are important
          if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
          else:
            continue

        for i in zip(sentence.split(), prediction, test_dataset["word_labels2"].iloc[tmp_num].split(',')):
            kws_pairs.append(i)

    predict_kws = []
    tmp_wd = []
    tmp_pred, tmp_gold = '', ''
    for word, pred, gold in kws_pairs:
        if pred != 'O':
            if len(tmp_wd) == 0: 
                tmp_pred = pred
            else:
                if tmp_pred != pred:
                    predict_kws.append((' '.join(tmp_wd), tmp_pred))
                    tmp_wd = []
                    tmp_pred = pred
            tmp_wd.append(word)
        else:
            if len(tmp_wd) > 0:
                predict_kws.append((' '.join(tmp_wd), tmp_pred))
                tmp_wd = []
                tmp_pred = pred

    # check whether there are new keywords
    gold_dict = build_gold_dict()
    new_kws, kw_tags = [], []
    gold_lists = list(gold_dict.keys())
    gold_lists = [g.lower() for g in gold_lists]
    for word, label in predict_kws:
        if not word.lower() in gold_lists:
            new_kws.append(word.lower())
            kw_tags.append(tag2cat[label])
    new_keywords = list(zip(new_kws, kw_tags))

    new_kws_dict, new_kws_pos = {}, []
    for kw, cat in new_keywords:
        pos = pos_tag(kw.split())
        if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
            key = ' '.join([p for p, t in pos])
            tag = [t for p, t in pos]
            new_kws_pos.append(key)
            if not key in new_kws_dict.keys():
                new_kws_dict[key] = (tag, cat)

    new_kws = list(new_kws_dict.keys())
    cats = [value[1] for value in new_kws_dict.values()]

    df = pd.DataFrame({"keyword": new_kws, "category": cats})

    s = df['keyword'].apply(lengthOfTokens)
    s.sort_values()

    predict_kws = Counter(predict_kws)
    new_kws_pos = Counter(new_kws_pos)

    predict_kws = sorted(predict_kws.items(), key=lambda pair: pair[1], reverse=True)
    new_kws_pos = sorted(new_kws_pos.items(), key=lambda pair: pair[1], reverse=True)

    return predict_kws, new_kws_pos, df

In [ ]:
predict_kws, new_kws_pos, new_kws_dict = countKeywords(data2, model2)

In [ ]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

def get_distribution(inputs, name):
    true_res = {}
    for k, v in tag2cat.items():
        indexlist = get_index_positions(inputs, k)
        left, right = 0, 0
        new_labels = []
        for i in indexlist:
            if left == 0:
                left = i
                right = i
                continue
            if right+1 == i:
                right = i
            else:
                new_labels.append((left, right))
                left = i
                right = i
        true_res[v] = new_labels
    print(f"The distribution of {name} is:")
    for k,v in true_res.items():
        print(f"There are {len(v)} occurances in {k}")
    return true_res

def get_confusion_matrix(true_labels, true_predictions):
    # strict mode
    tp_dict = {}
    for true_k, true_v in true_labels.items():
        true_total = len(true_v)
        tmp_dict = {}
        tmp_total = 0
        for pred_k, pred_v in true_predictions.items():
            num = 0
            for v in pred_v:
                if v in true_v:
                    num += 1
            tmp_dict[pred_k] = num
            tmp_total += num
        tmp_dict["O"] = true_total - tmp_total
        tp_dict[true_k] = tmp_dict
    tp_dict
    tmp_dict = {}
    for pred_k, pred_v in true_predictions.items():
        true_total = len(pred_v)
        num = 0
        for tp_k, tp_v in tp_dict.items():
            num += tp_v[pred_k]
        tmp_dict[pred_k] = true_total - num
    tmp_dict["O"] = 0
    tp_dict["O"] = tmp_dict
    return pd.DataFrame(tp_dict).T

In [ ]:
get_distribution(labels, "labels")

In [ ]:
true_labels = get_distribution(labels, "labels")
true_predictions = get_distribution(predictions, "predictions")
dff = get_confusion_matrix(true_labels, true_predictions)
dff

In [ ]:
import pickle

with open("predicted_keywords_ap.pkl", 'wb') as fh:
   pickle.dump(predict_kws, fh)

with open("newly_keywords_ap.pkl", 'wb') as fh:
   pickle.dump(new_kws_pos, fh)

new_kws_dict.to_csv("new_keywords_ap_07_18.csv")

In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_size = 0.8
train_dataset2 = data2.sample(frac=train_size,random_state=200)
test_dataset2 = data2.drop(train_dataset2.index).reset_index(drop=True)
train_dataset2 = train_dataset2.reset_index(drop=True)

print("FULL Dataset: {}".format(data2.shape))
print("TRAIN Dataset: {}".format(train_dataset2.shape))
print("TEST Dataset: {}".format(test_dataset2.shape))

training_set2 = dataset2(train_dataset2, tokenizer, MAX_LEN)
testing_set2 = dataset2(test_dataset2, tokenizer, MAX_LEN)

train_params2 = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params2 = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader2 = DataLoader(training_set2, **train_params2)
testing_loader2 = DataLoader(testing_set2, **test_params2)

model2 = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids2))
model2.to(device)

optimizer2 = torch.optim.Adam(params=model2.parameters(), lr=LEARNING_RATE)

trLosslist, trAcclist, evalLosslist, evalAcclist = [], [], [], []
# label_pred_dict = {}
# for epoch in range(EPOCHS):
#     print(f"Training epoch: {epoch + 1}")
#     tmp_dict = {}
#     epoch_loss, tr_accuracy = train2(epoch)
#     labels, predictions, eval_loss, eval_accuracy, eval_probability = valid(model2, testing_loader2)
#     trLosslist.append(epoch_loss)
#     trAcclist.append(tr_accuracy)
#     evalLosslist.append(eval_loss)
#     evalAcclist.append(eval_accuracy)
#     tmp_dict["labels"] = labels
#     tmp_dict["predictions"] = predictions
#     tmp_dict["probability"] = eval_probability
#     label_pred_dict[epoch] = tmp_dict

# resultdf = pd.DataFrame({"Epoch": list(range(1, EPOCHS+1)),
#                          "Train_loss": trLosslist,
#                          "Eval_loss": evalLosslist,
#                          "Train_Acc": trAcclist,
#                          "Eval_Acc": evalAcclist})
# resultdf    

In [ ]:
# !pip install pthflops
# from pthflops import count_ops

model2.train()
ids = batch['input_ids'].to(device, dtype = torch.long)
mask = batch['attention_mask'].to(device, dtype = torch.long)
labels = batch['labels'].to(device, dtype = torch.long)
count_ops(model2, ids)

In [ ]:
# !pip install thop-0.0.31.post2005241907-py3-none-any.whl
from thop import profile
ids = batch['input_ids'].to(device, dtype = torch.long)
mask = batch['attention_mask'].to(device, dtype = torch.long)
labels = batch['labels'].to(device, dtype = torch.long)
macs, params = profile(model2, inputs=(ids, mask), verbose=False)

In [ ]:
labels, predictions, eval_loss, eval_accuracy, eval_probability = valid(model2, testing_loader2)

In [ ]:
len(labels), len(predictions), len(eval_probability)

In [ ]:
labels1 = [i.cpu().detach().numpy() for i in labels]
predictions1 = [i.cpu().detach().numpy() for i in predictions]
eval_probability1 = [i.cpu().detach().numpy() for i in eval_probability]

In [ ]:
from sklearn.metrics import confusion_matrix, auc, roc_curve, roc_auc_score

def autodl_auc(outputs, targets, predictions):
    
    targets = np.asarray(targets)
    predictions = np.asarray(predictions)
    
    numclass = targets.max()+1
    
    boolean_array = np.zeros((len(outputs),numclass), dtype=bool)
    
    for labelindex in range(numclass):
        boolean_array[:,labelindex]= (targets == labelindex)
    
    
    auc = roc_auc_score(boolean_array, outputs)
    auc_1 = 2*auc-1
    
    
    return round(auc, 2), round(auc_1, 2)

autodl_auc(eval_probability1, labels1, predictions1)

In [ ]:
def plot_roc_curves(valid_outputs, valid_targets, valid_predictions):
    
    print("--------------------------------------------")
    print("ROC Curves")
    print("--------------------------------------------")
 
    valid_targets = np.asarray(valid_targets)
    valid_predictions = np.asarray(valid_predictions)
    valid_outputs = np.asarray(valid_outputs)

    numclass = valid_predictions.max()+1 
    
    valid_auc_curves = []

    for labelindex in range(numclass):
        valid_binary_targets = (valid_targets == labelindex)
        valid_binary_predictions = (valid_predictions == labelindex)
       
        selected_valid_outputs = valid_outputs[:,labelindex]  
        valid_fpr, valid_tpr, _ = roc_curve(valid_binary_targets, selected_valid_outputs)
        valid_auc_curves.append([valid_fpr,valid_tpr])  
    
     # Plot ROC Curves

    fig_height = 8 if numclass <= 5 else 16
    fig_width =  16 if numclass <= 5 else 30
    fig = plt.figure(figsize=(fig_width,fig_height))
    
    for idx, auc_cur in enumerate(valid_auc_curves): 
        plt.plot(auc_cur[0], auc_cur[1], marker='.',  label='Class:'+str(idx))
    plt.plot([0,1], [0,1], linestyle='--', color='black')
    plt.title('Valid ROC Curves - ', fontsize=20)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.legend()
    
    plt.pause(0.1)
    
plot_roc_curves(eval_probability1, labels1, predictions1)

In [ ]:
train_size = 0.8
train_dataset3 = data3.sample(frac=train_size,random_state=200)
test_dataset3 = data3.drop(train_dataset3.index).reset_index(drop=True)
train_dataset3 = train_dataset3.reset_index(drop=True)

print("FULL Dataset: {}".format(data3.shape))
print("TRAIN Dataset: {}".format(train_dataset3.shape))
print("TEST Dataset: {}".format(test_dataset3.shape))

training_set3 = dataset2(train_dataset3, tokenizer, MAX_LEN)
testing_set3 = dataset2(test_dataset3, tokenizer, MAX_LEN)

train_params3 = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params3 = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader3 = DataLoader(training_set3, **train_params3)
testing_loader3 = DataLoader(testing_set3, **test_params2)

In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 4
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_size = 0.8
train_dataset2 = data2.sample(frac=train_size,random_state=200)
test_dataset2 = data2.drop(train_dataset2.index).reset_index(drop=True)
train_dataset2 = train_dataset2.reset_index(drop=True)

print("FULL Dataset: {}".format(data2.shape))
print("TRAIN Dataset: {}".format(train_dataset2.shape))
print("TEST Dataset: {}".format(test_dataset2.shape))

training_set2 = dataset2(train_dataset2, tokenizer, MAX_LEN)
testing_set2 = dataset2(test_dataset2, tokenizer, MAX_LEN)

train_params2 = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params2 = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader2 = DataLoader(training_set2, **train_params2)
testing_loader2 = DataLoader(testing_set2, **test_params2)

model2 = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids2))
model2.to(device)

optimizer2 = torch.optim.Adam(params=model2.parameters(), lr=LEARNING_RATE)

epoch_loss_list, tr_accuracy_list, eval_loss_list, eval_accuracy_list = [], [], [], []
labels_list, predictions_list = [],[]
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    epoch_loss, tr_accuracy = train2(epoch)
    labels, predictions, eval_loss, eval_accuracy = valid(model2, testing_loader2)
    epoch_loss_list.append(epoch_loss) 
    tr_accuracy_list.append(tr_accuracy) 
    eval_loss_list.append(eval_loss) 
    eval_accuracy_list.append(eval_accuracy)

In [ ]:
loss_results = pd.DataFrame({'Epoch': list(range(1, EPOCHS+1)) * 2,
                            'Loss': epoch_loss_list + eval_loss_list,
                            'Label': ['Training'] * EPOCHS + ['Validation'] * EPOCHS})

acc_results = pd.DataFrame({'Epoch': list(range(1, EPOCHS+1)) * 2,
                            'Accuracy': tr_accuracy_list + eval_accuracy_list,
                            'Label': ['Training'] * EPOCHS + ['Validation'] * EPOCHS})


sns.set_theme(style='darkgrid', palette='deep', font='sans-serif', font_scale=1.3, rc={'figure.figsize':(20.7,8.27)})
fig, axs = plt.subplots(1, 2)
sns.lineplot(data=loss_results, x="Epoch", y='Loss', hue="Label", ax=axs[0])
plt.legend(labels=["Training loss", "Validation loss"])
sns.lineplot(data=acc_results, x="Epoch", y='Accuracy', hue="Label", ax=axs[1])
plt.legend(labels=["Training accuracy", "Validation accuracy"])

In [ ]:
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(
    fpr["micro"],
    tpr["micro"],
    label="micro-average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),
    color="deeppink",
    linestyle=":",
    linewidth=4,
)

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

Some extension of Receiver operating characteristic to multiclass

In [ ]:
!pip install seqeval
from seqeval.metrics import classification_report

labels = [labels]
predictions = [predictions]
print(classification_report(labels, predictions))

In [ ]:
labels, predictions, eval_loss, eval_accuracy = valid(model2, testing_loader3)

In [ ]:
!pip install seqeval
from seqeval.metrics import classification_report

labels = [labels]
predictions = [predictions]
print(classification_report(labels, predictions))

In [ ]:
model2.save_pretrained("./ftoken_model_epoch4")

In [ ]:
def lengthOfTokens(pair):
        return len(pair.split())

def countKeywords(test_dataset, model):
    kws_pairs = []
    for tmp_num in range(len(test_dataset)):
        sentence = test_dataset["sentence"].iloc[tmp_num]

        inputs = tokenizer(sentence.split(),
                            is_split_into_words=True, 
                            return_offsets_mapping=True, 
                            padding='max_length', 
                            truncation=True, 
                            max_length=MAX_LEN,
                            return_tensors="pt")

        # move to gpu
        ids = inputs["input_ids"].to(device)
        mask = inputs["attention_mask"].to(device)
        # forward pass
        outputs = model(ids, attention_mask=mask)
        logits = outputs.logits

        active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

        tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
        token_predictions = [ids_to_labels2[i] for i in flattened_predictions.cpu().numpy()]
        wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

        prediction = []
        for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
          #only predictions on first word pieces are important
          if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
          else:
            continue

        for i in zip(sentence.split(), prediction, test_dataset["word_labels2"].iloc[tmp_num].split(',')):
            kws_pairs.append(i)

    predict_kws = []
    tmp_wd = []
    tmp_pred, tmp_gold = '', ''
    for word, pred, gold in kws_pairs:
        if pred != 'O':
            if len(tmp_wd) == 0: 
                tmp_pred = pred
            else:
                if tmp_pred != pred:
                    predict_kws.append((' '.join(tmp_wd), tmp_pred))
                    tmp_wd = []
                    tmp_pred = pred
            tmp_wd.append(word)
        else:
            if len(tmp_wd) > 0:
                predict_kws.append((' '.join(tmp_wd), tmp_pred))
                tmp_wd = []
                tmp_pred = pred

    # check whether there are new keywords
    gold_dict = build_gold_dict()
    new_kws, kw_tags = [], []
    gold_lists = list(gold_dict.keys())
    gold_lists = [g.lower() for g in gold_lists]
    for word, label in predict_kws:
        if not word.lower() in gold_lists:
            new_kws.append(word.lower())
            kw_tags.append(tag2cat[label])
    new_keywords = list(zip(new_kws, kw_tags))

    new_kws_dict, new_kws_pos = {}, []
    for kw, cat in new_keywords:
        pos = pos_tag(kw.split())
        if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
            key = ' '.join([p for p, t in pos])
            tag = [t for p, t in pos]
            new_kws_pos.append(key)
            if not key in new_kws_dict.keys():
                new_kws_dict[key] = (tag, cat)

    new_kws = list(new_kws_dict.keys())
    cats = [value[1] for value in new_kws_dict.values()]

    df = pd.DataFrame({"keyword": new_kws, "category": cats})

#     s = df['keyword'].apply(lengthOfTokens)
#     df.sort_values(by=['keyword'], key=lambda x: lengthOfTokens(x))

    predict_kws = Counter(predict_kws)
    new_kws_pos = Counter(new_kws_pos)

    predict_kws = sorted(predict_kws.items(), key=lambda pair: pair[1], reverse=True)
    new_kws_pos = sorted(new_kws_pos.items(), key=lambda pair: pair[1], reverse=True)

    return predict_kws, new_kws_pos, df

In [ ]:
predict_kws, new_kws_pos, new_kws_dict = countKeywords(test_dataset2, model2)

In [ ]:
new_kws_dict

In [ ]:
with open("predicted_keywords_ftoken_upgrade1.pkl", 'wb') as fh:
   pickle.dump(predict_kws, fh)

with open("newly_keywords_ftoken_upgrade1.pkl", 'wb') as fh:
   pickle.dump(new_kws_pos, fh)

new_kws_dict.to_csv("new_keywords_ftoken_07_07.csv")

## LSTM code checking

In [1]:
import ast
import pandas as pd
import torch
import nltk
from nltk import pos_tag
import numpy as np
nltk.download('averaged_perceptron_tagger')
from collections import Counter

tag_dict = {}
tag_dict['Sustainability preoccupations'] = 'I-sus'
tag_dict['Digital transformation'] = 'I-dig'
tag_dict['Change in management'] = 'I-mag'
tag_dict['Innovation activities'] = 'I-inn'
tag_dict['Business Model'] = 'I-bus'
tag_dict['Corporate social responsibility ou CSR'] = 'I-cor'
tag_dict['marco-label'] = 'I-mar'
tag2cat = {v: k for k, v in tag_dict.items()}

labels_to_ids2 = {'O':0, 'I-sus':1, 'I-dig':2, 'I-mag':3, 'I-inn':4, 'I-bus':5, 'I-cor':6, 'I-mar':7}
ids_to_labels2 = {v: k for k, v in labels_to_ids2.items()}

def build_gold_dict():
    # read from excel
    read_file = pd.read_excel('../input/gold-dict/Terms malantin 1er juin 2022.xlsx',
                              sheet_name='categories 1 juin 2022')
    read_file.dropna(0, how='all', inplace=True)
    read_file.dropna(1, how='all', inplace=True)

    gold_dict = {}
    for i in range(1, len(read_file)):
        if read_file.iloc[i]['Main form'] is None:
            continue
        gold_dict[read_file.iloc[i]['Main form'].strip()] = []

    for i in range(1, len(read_file)):
        index = read_file.iloc[i]['Main form'].strip()
        if index is None:
            continue
        if not read_file.iloc[i].isna()['Sustainability preoccupations']:
            gold_dict[index].append('Sustainability preoccupations')
        if not read_file.iloc[i].isna()['Digital transformation']:
            gold_dict[index].append('Digital transformation')
        if not read_file.iloc[i].isna()['Change in management']:
            gold_dict[index].append('Change in management')
        if not read_file.iloc[i].isna()['Innovation activities']:
            gold_dict[index].append('Innovation activities')
        if not read_file.iloc[i].isna()['Business Model']:
            gold_dict[index].append('Business Model')
        if not read_file.iloc[i].isna()['Corporate social responsibility ou CSR']:
            gold_dict[index].append('Corporate social responsibility ou CSR') 

    # Change the category for four keywords
    # academic institutions, university & research institutions, service among university, 
    # worldwide research centres
    category = 'Innovation activities'
    changelist = ['academic institutions', 'worldwide research centers', 'university and research institutions', 'customer service among university']
    for c in changelist:
        gold_dict[c][0] = category

    # Deal with the singular and plural cases in keywords
    cortext3 = open("../input/cortext/Cortext3_min_delac_flex_utf8.txt", "r")
    lines = cortext3.readlines()
    lefts, rights = [], []
    for line in lines:
        left, right = line.split(',')
        if left != right.split('.')[0]:
            lefts.append(left)
            rights.append(right.split('.')[0])

    tmp_gold_dict = gold_dict.copy()
    for key, value in tmp_gold_dict.items():
        if key in lefts:
            index = lefts.index(key)
            right = rights[index]
            gold_dict[right] = gold_dict[key]
        if key in rights:
            indices = [i for i, word in enumerate(rights) if word == key]
            for index in indices:
                left = lefts[index]
                gold_dict[left] = gold_dict[key]

    tmp_dict = {}
    for k in gold_dict.keys():
        tmp_dict[k] = []
    for k, v in gold_dict.items():
        tmp_dict[k] = list(set(v))
    gold_dict = tmp_dict

    return gold_dict

def tags_to_keywords(sample, words):
    indices = [i for i, l in enumerate(sample) if l != 'O']
    keywords, key_cats = [], []
    for j, id in enumerate(indices):
        if j == 0:
            start = end = id
            continue
        if j == len(indices):
            pos = pos_tag(words[start:end+1])
            if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
                keywords.append(' '.join(words[start:end+1]))
                key_cats.append((' '.join(words[start:end+1]), sample[start:end+1]))
            continue
        if end+1 == id:
            end = id
        else:
            try: # the index is out of the words
                pos = pos_tag(words[start:end+1])
                if (pos[-1][1] == 'NN' or pos[-1][1] == 'NNS' or pos[-1][1] == 'NNP' or pos[-1][1] == 'NNPS') and pos[0][1] != 'CC':
                    keywords.append(' '.join(words[start:end+1]))
                    key_cats.append((' '.join(words[start:end+1]), sample[start:end+1]))
                start = end = id
            except:
                continue
    return list(set(keywords)), key_cats

def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

def get_distribution(inputs, name):
    true_res = {}
    for k, v in tag2cat.items():
        indexlist = get_index_positions(inputs, k)
        left, right = 0, 0
        new_labels = []
        for i in indexlist:
            if left == 0:
                left = i
                right = i
                continue
            if right+1 == i:
                right = i
            else:
                new_labels.append((left, right))
                left = i
                right = i
        true_res[v] = new_labels
    print(f"The distribution of {name} is:")
    for k,v in true_res.items():
        print(f"There are {len(v)} occurances in {k}")
    return true_res

def get_confusion_matrix(true_labels, true_predictions):
    # strict mode
    tp_dict = {}
    for true_k, true_v in true_labels.items():
        true_total = len(true_v)
        tmp_dict = {}
        tmp_total = 0
        for pred_k, pred_v in true_predictions.items():
            num = 0
            for v in pred_v:
                if v in true_v:
                    num += 1
            tmp_dict[pred_k] = num
            tmp_total += num
        tmp_dict["O"] = true_total - tmp_total
        tp_dict[true_k] = tmp_dict
    tp_dict
    tmp_dict = {}
    for pred_k, pred_v in true_predictions.items():
        true_total = len(pred_v)
        num = 0
        for tp_k, tp_v in tp_dict.items():
            num += tp_v[pred_k]
        tmp_dict[pred_k] = true_total - num
    tmp_dict["O"] = 0
    tp_dict["O"] = tmp_dict
    return pd.DataFrame(tp_dict).T

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def get_recall(Labels, Predictions, ignore=True):
    recalls = []
    for i, lbs in enumerate(Labels):
        preds = Predictions[i]
        if not lbs:
            if not ignore:
                recalls.append(1.0 if not preds else 0.0)
            continue
        recalls.append(np.nan_to_num(len(intersection(preds, lbs)) / len(lbs)))
    return np.mean(recalls)

def get_precision(Labels, Predictions, ignore=True):
    precisions = []
    for i, preds in enumerate(Predictions):
        lbs = Labels[i]
        if not preds:
            if not ignore:
                precisions.append(1.0 if not preds else 0.0)
            continue
        precisions.append(np.nan_to_num(len(intersection(preds, lbs)) / len(preds)))
    return np.mean(precisions)

def get_f1score(Labels, Predictions, ignore=True):
    precision = get_precision(Labels, Predictions, ignore)
    recall = get_recall(Labels, Predictions, ignore)
    return np.nan_to_num(2 * precision * recall / (precision + recall))

def lengthOfTokens(pair):
    return len(pair.split())

def tokenize_example(example, max_length=300):
    return example.split(' ')[:max_length]

def split_tags(example, max_length=300):
    return example.split(',')[:max_length]

def argmax(x):
    # return the argmax as a python int
    _, idx = torch.max(x, 1)
    return idx.item()

def log_sum_exp(x):
    #return log sum exp of a tensor
    m = torch.max(x, -1)[0]
    return m + torch.log(torch.sum(torch.exp(x - m.unsqueeze(-1)), -1))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
